### 1. Import and Load Packages 

In [ ]:
# Load packages
import pandas as pd
import numpy as np
from datetime import date
import csv
import re
import datetime
import os
from sklearn.feature_selection import VarianceThreshold
from pandas import read_csv
from sklearn.datasets import load_breast_cancer
from datetime import date
import kaplanmeier as km
from sklearn.decomposition import PCA
import pylab as pl
from itertools import cycle
import itertools as it
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
%matplotlib inline
from qqman import qqman
from pandas_plink import read_plink
from scipy.stats import ttest_ind
import math
import os
from lifelines import KaplanMeierFitter
from pandas import DataFrame
import pandas as pd
import seaborn as sns
sns.set(color_codes=True)
import numpy as np
from pandas.plotting import scatter_matrix
from sklearn import preprocessing
from numpy import set_printoptions
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import datasets
import seaborn as sns; sns.set()
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
#from xgboost import XGBClassifier
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import ElasticNet
from numpy import arange
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
import pandas as pd
from google.cloud import bigquery
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegressionCV

import os
import matplotlib.pyplot as plt
import textwrap
import re
from tqdm import tqdm
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn import metrics
from pprint import pprint

### 2. Load Exposure Cohort from BigQuery

##### exp_2022

In [ ]:
user_id = 'kbechler'
nero_gcp_project = 'som-nero-nigam-starr'
cdm_project_id = 'som-nero-nigam-starr'
cdm_dataset_id = 'starr_omop_cdm5_deid_20211213'
work_project_id = 'som-nero-nigam-starr'
work_dataset_id = f'{user_id}_explore'
cdm_subset_dataset_id = 'cdm_subset'

Set up connection.

In [ ]:
os.getcwd()

In [ ]:
# Use correct path whether you are local or Nero
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/katelyn/.config/gcloud/legacy_credentials/kbechler@stanford.edu/adc.json'  

# Set correct Nero project
os.environ['GCLOUD_PROJECT'] = nero_gcp_project
client = bigquery.Client(project = work_project_id)

In [ ]:
# Import exposure cohort
sql = "select * from som-nero-nigam-starr.kbechler_explore.exp_2022"
exp_df = client.query(sql).to_dataframe()

In [ ]:
exp_df.head()

### 3. Bring in Demographic Features 

In [ ]:
os.getcwd()

In [ ]:
# Person table
person = pd.read_csv('person.tsv', sep = '\t')
print(len(person))
person.head()

In [ ]:
# Feature columns to bring in 
features = ['person_id','_gender_name', 'birth_DATETIME', 'race_concept_id', 'race_source_value', 'ethnicity_concept_id', 
           'ethnicity_source_value']

In [ ]:
# Subset dataframe
person_features = person[features]

In [ ]:
print(len(person_features))
person_features.head()

In [ ]:
print("Number of blank race:", sum(person_features.race_concept_id == 0))
print("Number of blank ethnicity:", sum(person_features.ethnicity_concept_id == 0))

In [ ]:
person_features.birth_DATETIME = pd.to_datetime(person_features.birth_DATETIME)

In [ ]:
# Merge person_features with cohort df
patient_df = pd.merge(exp_df, person_features, left_on = 'subject_id', right_on = 'person_id', how = 'inner' )

In [ ]:
# Convert cohort start date to datetime
patient_df.cohort_start_date = pd.to_datetime(patient_df.cohort_start_date)

In [ ]:
# Calculate age at SLE diagnosis
patient_df['age_at_diagnosis'] = patient_df.cohort_start_date - patient_df.birth_DATETIME
patient_df.age_at_diagnosis = pd.to_numeric(patient_df.age_at_diagnosis.dt.days, downcast = 'integer')

In [ ]:
patient_df.age_at_diagnosis = patient_df.age_at_diagnosis/365

In [ ]:
# Change name of birth date column and _gender_name column
patient_df = patient_df.rename(columns={'birth_DATETIME': 'birth_date', '_gender_name':'gender'})

In [ ]:
# Convert gender to numerical categorical variable
# 1: Female
# 2: Male
patient_df.gender = np.where(patient_df.gender == 'FEMALE', 1, 2)

In [ ]:
patient_df = patient_df[patient_df.age_at_diagnosis >= 14]
print("Total number of patients:", len(patient_df))

##### Race and Ethnicity

In [ ]:
# 8515 == Asian
# 8527 == White
# 2000039205 == Other
# 8516 == Black
# 2000039212 == Unknown
# 8557 == Native Hawaiian or Other Pacific Islander
# 2000039210 == Patient Refused
# 8657 == American Indian or Alaska Native
# 0 == Unknown
# 2000039200 == Unknown

In [ ]:
patient_df['race_final'] = np.where((patient_df.race_concept_id == 2000039200) | 
                                     (patient_df.race_concept_id == 0) |
                                    (patient_df.race_concept_id == 2000039210)|
                                    (patient_df.race_concept_id == 2000039212) |
                                    (patient_df.race_concept_id == 2000039205 ), 'Unknown', patient_df.race_concept_id)

In [ ]:
patient_df.race_final = np.where(patient_df.race_concept_id == 8515, 'Asian', 
                                 np.where(patient_df.race_concept_id == 8527, 'White', 
                                         np.where(patient_df.race_concept_id == 8516, 'Black', 
                                                 np.where(patient_df.race_concept_id == 8557, 
                                                         'Native Hawaiian or Other Pacific Islander', 
                                    np.where(patient_df.race_concept_id == 8657, 'American Indian or Alaska Native',
                                            patient_df.race_final)))))

In [ ]:
patient_df.race_final.unique()

In [ ]:
# Convert to categorical 
# Label encoding
patient_df['race_binned'] = np.where(patient_df.race_final == 'Asian', 1, patient_df.race_final)
patient_df['race_binned'] = np.where(patient_df.race_final == 'White', 2, patient_df.race_binned)
patient_df['race_binned'] = np.where(patient_df.race_final == 'Black', 3, patient_df.race_binned)
patient_df['race_binned'] = np.where(patient_df.race_final == 'Native Hawaiian or Other Pacific Islander',
                                     4, patient_df.race_binned)
patient_df['race_binned'] = np.where(patient_df.race_final == 'American Indian or Alaska Native', 
                                     5, patient_df.race_binned)
patient_df['race_binned'] = np.where(patient_df.race_final == 'Unknown', 0, patient_df.race_binned)

In [ ]:
patient_df.race_binned.unique()

In [ ]:
patient_df.drop(columns = ['race_source_value', 'race_concept_id'], axis = 1, inplace  = True)

In [ ]:
patient_df.head()

In [ ]:
# 38003564 == Non-Hispanic/Non-Latino
# 0 = Unknown
# 38003563 == Hispanic/Latino

In [ ]:
patient_df['ethnicity_final'] = np.where(patient_df.ethnicity_concept_id == 38003563, 'Hispanic/Latino', 
                                        np.where(patient_df.ethnicity_concept_id ==38003564, 
                                                'Non-Hispanic/Non-Latino', 'Unknown'))

In [ ]:
patient_df['ethnicity_binned'] = np.where(patient_df.ethnicity_final == 'Unknown', 0, patient_df.ethnicity_final)
patient_df.ethnicity_binned = np.where(patient_df.ethnicity_binned == 'Hispanic/Latino',1, patient_df.ethnicity_binned)
patient_df.ethnicity_binned = np.where(patient_df.ethnicity_binned == 'Non-Hispanic/Non-Latino', 
                                       2, patient_df.ethnicity_binned)

In [ ]:
patient_df.ethnicity_binned.unique()

In [ ]:
patient_df.drop(columns = ['ethnicity_concept_id', 'ethnicity_source_value'], axis = 1,  inplace = True)

In [ ]:
patient_df.head()

In [ ]:
patient_df.head()

In [ ]:
len(patient_df)

In [ ]:
# Normalize age?

### 4. Bring in Engineered Feature IDs 

https://docs.google.com/spreadsheets/d/1BJ6NkCTlnQZb8VsYwb0ucfLLkQLUuY-1VuzM7tEaecs/edit#gid=0

In [ ]:
# Read in excel file of curated features from Liya
features_file = pd.read_csv('feature_ids_2022.csv', sep = ',')

In [ ]:
print(len(features_file))
features_file.head()

### 5. Bring in Condition Occurrence Features

Feature is count of condition occurrences per patient prior to SLE diagnosis. Could also explore being binary variable of present or not present.

In [ ]:
# Condition occurrence table
cond_occ = pd.read_csv('condition_occurrence.tsv', sep='\t')
print(len(cond_occ))
cond_occ.head()

In [ ]:
# Subset to confirm only bringing in conditions prior to cohort start date
# Using less than becuase the date includes all medical records of that diagnosis
cond_occ = pd.merge(cond_occ, patient_df[['person_id', 'cohort_start_date']], on = 'person_id', how = 'left')
cond_occ.condition_start_DATE = pd.to_datetime(cond_occ.condition_start_DATE)
cond_occ = cond_occ[cond_occ.condition_start_DATE < cond_occ.cohort_start_date]
print(len(cond_occ))

In [ ]:
# Subset to appropriate columns
features = ['person_id', '_condition_name', 'condition_concept_id']
cond_occ_features = cond_occ[features]
cond_occ_features['count'] = 1

In [ ]:
# Bring in condition features from features file
condition_features = list(features_file[features_file.table == 'condition'].feature_liya.unique())
print(condition_features)

In [ ]:
cond_occ_features['labels']  = np.nan

In [ ]:
feature_concepts = {}
def get_concept_ids(feature_list):
    for feature in feature_list:
        concept_ids = features_file[features_file.feature_liya == feature].concept_id
        feature_concepts[feature] = concept_ids
        cond_occ_features['labels'] = np.where(cond_occ_features.condition_concept_id.isin(concept_ids), feature,
                                              cond_occ_features.labels)
get_concept_ids(condition_features)
cond_occ_features.head()

In [ ]:
# Subset to patients in cohort
cond_occ_features = cond_occ_features[cond_occ_features.person_id.isin(exp_df.subject_id)]
print("Number of patients with conditions present:", len(cond_occ_features.person_id.unique()))

In [ ]:
# Transform from long to wide dataframe and count condition concept ids
# Fill blank values with 0
cond_occ_features_wide = cond_occ_features.pivot_table(index = 'person_id', 
                                                      columns = ['labels'], 
                                                      values = 'count',
                                                      aggfunc = 'any',
                                                      fill_value = 0).astype(int).reset_index()

In [ ]:
# Drop nan column
cond_occ_features_wide.drop('nan', axis = 1, inplace = True)

In [ ]:
cond_occ_features_wide.describe()

##### Merge with patient dataframe 

In [ ]:
df = pd.merge(patient_df, cond_occ_features_wide, left_on = 'subject_id', right_on = 'person_id', how = 'left')
df.drop(['person_id_x', 'person_id_y'], axis = 1, inplace = True)
print(len(df))

### 6. Bring in Drug Features 

In [ ]:
# Drug table
drugs = pd.read_csv('drug_era.tsv', sep = '\t')
print("Number of patients in drug table:", drugs.person_id.nunique())
drugs.head()

In [ ]:
# Subset to drugs prior to cohort start date
drugs = pd.merge(drugs, patient_df[['person_id', 'cohort_start_date']], on = 'person_id', how = 'left')
drugs.drug_era_start_DATE = pd.to_datetime(drugs.drug_era_start_DATE)
drugs = drugs[drugs.drug_era_start_DATE < drugs.cohort_start_date]
print(len(drugs))

In [ ]:
# Subset to appropriate columns
features = ['person_id', '_drug_name', 'drug_concept_id', 'drug_exposure_count']
drugs_features = drugs[features]
drugs_features['labels'] = np.nan

# Bring in drugs features from features file
drugs_list = list(features_file[(features_file.table == 'drug') & (features_file.include == 'yes')].feature_liya.unique())
print(drugs_list)

In [ ]:
feature_concepts = {}
def get_concept_ids(feature_list):
    for feature in feature_list:
        concept_ids = features_file[features_file.feature_liya == feature].concept_id
        feature_concepts[feature] = concept_ids
        drugs_features['labels'] = np.where(drugs_features.drug_concept_id.isin(concept_ids), feature,
                                              drugs_features.labels)

get_concept_ids(drugs_list)
drugs_features.head()

In [ ]:
drugs_features.labels.unique()

In [ ]:
# Subset to patients in cohort
drugs_features = drugs_features[drugs_features.person_id.isin(exp_df.subject_id)]
print("Number of patients with drugs:", drugs_features.person_id.nunique())

Drugs variables present or not for each drug for each patient id

In [ ]:
# Transform from long to wide dataframe and count drug concept_ids
# Fill blank values with 0
drugs_features_wide = drugs_features.pivot_table(index = 'person_id', 
                                                columns = ['labels'], 
                                                values = 'drug_exposure_count',
                                                aggfunc = 'any', 
                                                fill_value = 0).astype(int).reset_index()

In [ ]:
drugs_features_wide.head()

In [ ]:
drugs_features_wide.drop('nan', axis = 1, inplace = True)

In [ ]:
drugs_features_wide.person_id.nunique()

In [ ]:
drugs_features_wide.describe()

##### Merge with patient dataframe 

In [ ]:
df1 = pd.merge(df, drugs_features_wide, left_on = 'subject_id', right_on = 'person_id', how = 'left')

In [ ]:
df1.head()

In [ ]:
drug_list = list(drugs_features.labels.unique())

In [ ]:
drug_list.remove('nan')

In [ ]:
# Fill in NaN with 0 for persons missing drugs from drug exposure table
for drug in drug_list:
    df1[drug] = df1[drug].fillna(0)

In [ ]:
# Fill in NaN with 0 for persons missing conditions from condition table
cond_list = list(cond_occ_features.labels.unique())

In [ ]:
cond_list.remove('nan')

In [ ]:
for cond in cond_list:
    df1[cond] = df1[cond].fillna(0)

In [ ]:
df1.head()

In [ ]:
df1.sum()

### 7. Bring in Measurement Features 

In [ ]:
# Measurement table
measurement = pd.read_csv('measurement.tsv', sep = '\t')

In [ ]:
print(len(measurement))
measurement.head()

In [ ]:
print("Number of NA values in measurement values:", sum(measurement.value_as_number.isna()))

In [ ]:
# Remove rows with NA measurement values
measurement = measurement[measurement.value_as_number.notna()]
print("Number of populated values in measurement table:", len(measurement))

In [ ]:
# Subset to prior to cohort start date
measurement = pd.merge(measurement, patient_df[['person_id', 'cohort_start_date']], on = 'person_id', how = 'left')
measurement.measurement_DATE = pd.to_datetime(measurement.measurement_DATE)
measurement = measurement[measurement.measurement_DATE < measurement.cohort_start_date]
print("Number of populated values prior to index date:", len(measurement))

In [ ]:
print("Number of patients with measurement values:", measurement.person_id.nunique())

In [ ]:
# Confirm no null values
print("Number of null values:", sum(measurement.value_as_number.isnull()))

In [ ]:
# Subset to appropriate columns
measurement_features = measurement[['person_id', 'measurement_concept_id', 'value_as_number', '_measurement_name',
                                   'measurement_DATETIME', 'unit_source_value']]

# List out measurement features 
measurement_list = list(features_file[features_file.table == 'measurement'].feature_liya.unique())
measurement_features['labels'] = np.nan
print(measurement_list)

In [ ]:
feature_concepts = {}
def get_concept_ids(feature_list):
    for feature in feature_list:
        concept_ids = features_file[features_file.feature_liya == feature].concept_id
        feature_concepts[feature] = concept_ids
        measurement_features['labels'] = np.where(measurement_features.measurement_concept_id.isin(concept_ids), feature,
                                              measurement_features.labels)
        
get_concept_ids(measurement_list)

# Remove nan values
measurement_features = measurement_features[measurement_features.labels != 'nan']

# List out features populated
measurement_features.labels.unique()

In [ ]:
# Subset to patients in cohort
measurement_features = measurement_features[measurement_features.person_id.isin(exp_df.subject_id)]
print(measurement_features.person_id.nunique())

In [ ]:
# Look at units
units = measurement_features[['labels','_measurement_name', 'measurement_concept_id', 
                              'unit_source_value']].drop_duplicates()
units = units[units.unit_source_value.notnull()]
units.sort_values('labels', ascending = True).head()

In [ ]:
# For each person_id, concept_id combination take the row with the max datetime
max_date = measurement_features.groupby(['person_id', 'measurement_concept_id', '_measurement_name', 'labels'], 
                                       as_index = False).agg(
{'measurement_DATETIME': max})
print("Number of unique measurements based on concept id:", len(max_date))

In [ ]:
# Merge back with value and labels
max_date = pd.merge(max_date, measurement_features[['person_id', 'value_as_number','measurement_concept_id', 
                    'measurement_DATETIME', 'unit_source_value']], on = ['person_id',  'measurement_concept_id', 'measurement_DATETIME'], 
                   how = 'left')
max_date.head()

In [ ]:
qc = max_date.groupby(['person_id', 'measurement_concept_id'], as_index = False).agg({
    '_measurement_name':'count'
})
qc[qc._measurement_name == 4]

In [ ]:
max_date[max_date.person_id == 31419496]

In [ ]:
max_date.unit_source_value = str(max_date.unit_source_value)

max_date = max_date.groupby(['person_id', 'measurement_concept_id', '_measurement_name', 'labels',
                            'measurement_DATETIME'], as_index = False).agg({
    'value_as_number':'mean', 
    'unit_source_value':lambda x: ' '.join(x)
})

print("Number of patients with measurements:", len(max_date.person_id.unique()))

##### Explore measurement values 

##### Antibody DNA

Binary variable. Specific type of ANA antibody present in ~30% of people with SLE. Less than 1% of healthy individuals have antibody.

1: Present  
0: Not present

In [ ]:
max_date[max_date.labels == 'anti_double_stranded_DNA_antibody'].value_as_number.describe()

In [ ]:
units[units.labels == 'anti_double_stranded_DNA_antibody']

In [ ]:
dna_antibody = max_date[max_date.labels == 'anti_double_stranded_DNA_antibody']
dna_antibody = dna_antibody.pivot_table(index = 'person_id', 
                                                columns = ['labels'], 
                                                values = 'value_as_number',
                                                aggfunc = 'any', 
                                                fill_value = 0).astype(int).reset_index()
dna_antibody.head()

Units in IU/mL

In [ ]:
# Merge with patient df
df1.drop('person_id', inplace = True, axis = 1)
df1 = df1.rename(columns = {'subject_id': 'person_id'})
df2 = pd.merge(df1, dna_antibody, on = 'person_id', how = 'left')
df2.head()

##### C3 and C4 features

Low C3 (abnormal < 86 mg/dl, normal => 86 mg/dl, unknown NA) and low C4 (abnormal <= 20 mg/dl, normal > 20mg/dl, unknown NA) levels occur in active lupus. Categorical variable if both or one level is low. Mean of all C3 and C4 measurements taken.

C3/C4

1 Abnormal  
0 Normal  
-1 NA  

In [ ]:
c3_names = features_file[(features_file.feature_liya == 'c3')].name_starr_omop
c4_names = features_file[(features_file.feature_liya == 'c4')].name_starr_omop
c3 = max_date[max_date._measurement_name.isin(c3_names)]
c4 = max_date[max_date._measurement_name.isin(c4_names)]

c3_values = c3.pivot(index = 'person_id', 
                                 columns = ['_measurement_name'], 
                                 values = 'value_as_number').reset_index()
c3_values.describe()

In [ ]:
print("Number of patients with c3 values populated:", c3_values.person_id.nunique())

Can remove column with 'Complement C3a'. Unit source value for C3 is mg/dL.

In [ ]:
c3 = c3[c3._measurement_name != 'Complement C3a [Mass/volume] in Serum or Plasma']
c3_values = c3.pivot_table(index = 'person_id',
                          columns = ['_measurement_name'], 
                          values = 'value_as_number').reset_index()

c3_values.describe()
c3_values.columns = ['person_id', 'c3_final']

In [ ]:
units[units.labels == 'c3']

abnormal < 86 mg/dl, normal => 86 mg/dl, unknown NA

##### c4 features

abnormal <= 20 mg/dl, normal > 20mg/dl

In [ ]:
c4_values = c4.pivot_table(index = 'person_id', 
                                 columns = ['_measurement_name'], 
                                 values = 'value_as_number').reset_index()
print(len(c4_values))
print(len(c4_values.person_id.unique()))
c4_values.columns = ['person_id', 'c4_final']

Can also remove C4a column. Unit source value is mg/dL.

In [ ]:
# Merge with patient df
df2 = pd.merge(df2, c3_values, on = 'person_id', how = 'left')
df2 = pd.merge(df2, c4_values, on = 'person_id', how = 'left')

##### erythrocyte_sedimentation_rate feature

Lab test marker for inflammation. Faster than normal rate may indicate a systemic disease. ESR is 0–14 mm/h for males under 50 years old, 0–19 mm/h for females under 50 years old, 0–19 for males between 50 and 80 years old, and 0–29 mm/h for females between 50 and 80 years old.

1: Abnormal  
0: Normal  
-1: NA

In [ ]:
# Explore values
esr_names = features_file[features_file.feature_liya == 'erythrocyte_sedimentation_rate'].name_starr_omop
esr = max_date[max_date._measurement_name.isin(esr_names)]
esr_values = esr.pivot(index = 'person_id', 
                                       columns = ['_measurement_name'], 
                                       values = ['value_as_number', 'measurement_DATETIME']
                            ).reset_index()
esr_values.describe()

In [ ]:
print(len(esr_values))

In [ ]:
esr_values.columns = esr_values.columns.droplevel()

Can remove esr by Westergren method. Units = mm/h

In [ ]:
esr_values.drop('Erythrocyte sedimentation rate by Westergren method', axis = 1, inplace = True)
esr_values.columns = ['person_id', 'esr_rate', 'esr_rate_measurement', 'esr_rate_date', 'esr_rate_measurement_date']
esr_values.head()

In [ ]:
esr_values['esr_final'] = np.nan
esr_values.esr_final = np.where(esr_values.esr_rate_measurement.isna(),esr_values.esr_rate, esr_values.esr_final)
print(sum(esr_values.esr_final.isna()))

In [ ]:
esr_values.esr_final = np.where(esr_values.esr_rate.isna(),
                                esr_values.esr_rate_measurement, esr_values.esr_final)
print(sum(esr_values.esr_final.isnull()))

In [ ]:
esr_values.esr_rate_date = pd.to_datetime(esr_values.esr_rate_date)
esr_values.esr_rate_measurement_date = pd.to_datetime(esr_values.esr_rate_measurement_date)
sum(esr_values.esr_final.isnull())

In [ ]:
esr_values.esr_final = np.where((esr_values.esr_final.isna())&

                                (esr_values.esr_rate_date > esr_values.esr_rate_measurement_date),
                                esr_values.esr_rate, esr_values.esr_final)
esr_values.esr_final = np.where(esr_values.esr_final.isna(), esr_values.esr_rate_measurement, esr_values.esr_final)

In [ ]:
units[units.labels == 'erythrocyte_sedimentation_rate']

In [ ]:
esr_values = esr_values[esr_values.esr_final.notna()]

In [ ]:
esr_values.head()

In [ ]:
print("Number of patients with esr values:", len(esr_values.person_id.unique()))

In [ ]:
# Merge with patient df
df2 = pd.merge(df2, esr_values[['person_id', 'esr_final']], on = 'person_id',  how = 'left')

##### c_reactive feature 

Lab test marker for inflammation. abnormal if >= 0.5 mg/dl or 5mg/L, normal if below, NA if unknown. Categorical variable.

1: Abnormal (>5 mg/L, > 0.5 mg/dl)  
0: Normal (<5 mg/L, > 0.5 mg/dl)  
-1: NA

In [ ]:
units[units.labels == 'c_reactive']

In [ ]:
c_reactive_names = features_file[features_file.feature_liya == 'c_reactive'].name_starr_omop
c_reactive = max_date[max_date._measurement_name.isin(c_reactive_names)]
c_reactive_values = c_reactive.pivot(index = 'person_id', 
                                       columns = ['_measurement_name'], 
                                       values = 'value_as_number').reset_index()
c_reactive_values.describe()

In [ ]:
c_reactive = c_reactive[['person_id', 'measurement_concept_id', '_measurement_name', 'labels', 
                        'measurement_DATETIME', 'value_as_number']]
c_reactive = pd.merge(c_reactive, measurement_features[['person_id', 'measurement_concept_id', 'value_as_number', 
                                                       'measurement_DATETIME', 'unit_source_value']], 
                     on = ['person_id', 'measurement_concept_id', 'value_as_number', 'measurement_DATETIME'], 
                     how = 'left').drop_duplicates()
print(len(c_reactive))

In [ ]:
c_reactive.unit_source_value.unique()

In [ ]:
c_reactive[c_reactive.unit_source_value.isnull()]

In [ ]:
c_reactive.unit_source_value = np.where(c_reactive.unit_source_value.isnull(), 'mg/dL', c_reactive.unit_source_value)
c_reactive.unit_source_value = np.where(c_reactive.unit_source_value == 'mg/dl', 'mg/dL', c_reactive.unit_source_value)
c_reactive.unit_source_value = np.where(c_reactive.unit_source_value == 'MG/L', 'mg/L', c_reactive.unit_source_value)
c_reactive.unit_source_value = np.where(c_reactive.unit_source_value == 'mg/L (See scan or EMR data for detail)',
                                       'mg/L', c_reactive.unit_source_value)

In [ ]:
test = c_reactive.groupby(['unit_source_value'], as_index = False).agg({
    'person_id': 'nunique', 
    'value_as_number': ['mean', 'min', 'max']
})

In [ ]:
test.head()

In [ ]:
# Convert mg/L to mg/L
c_reactive['value_final'] = np.where(c_reactive.unit_source_value == 'mg/L', c_reactive.value_as_number/10, 
                                    c_reactive.value_as_number)

In [ ]:
c_reactive.value_final.describe()

Only one column with data. Some units are mg/L, some are mg/dl. Convert all to mg/dl in 'value_final'

In [ ]:
c_reactive_values = c_reactive.pivot(index = 'person_id', 
                                       columns = ['_measurement_name'], 
                                       values = ['value_final', 'measurement_DATETIME']).reset_index()
c_reactive_values.columns = c_reactive_values.columns.droplevel()
c_reactive_values.columns = ['person_id', 'c_reactive1', 'c_reactive2', 'c_reactive3', 'c_reactive1_date', 
                     'c_reactive2_date', 'c_reactive3_date']
c_reactive_values.head()

In [ ]:
c_reactive_values['c_reactive_final'] = np.nan

In [ ]:
c_reactive_values['c_reactive_final'] = np.where((c_reactive_values.c_reactive3.isnull())&(c_reactive_values.c_reactive2.isnull()), 
                                                 c_reactive_values.c_reactive1, c_reactive_values.c_reactive_final)

c_reactive_values.c_reactive_final = np.where((c_reactive_values.c_reactive_final.isnull())&(c_reactive_values.c_reactive2.isnull()), 
                                             c_reactive_values.c_reactive3, c_reactive_values.c_reactive_final)

c_reactive_values.c_reactive_final = np.where(c_reactive_values.c_reactive_final.isnull(), 
                                              c_reactive_values.c_reactive2, c_reactive_values.c_reactive_final)

In [ ]:
sum(c_reactive_values.c_reactive_final.isnull())

In [ ]:
print(len(c_reactive_values))
print("Number of patients with c_reactive values:", len(c_reactive_values.person_id.unique()))

In [ ]:
# Merge with patient df
df2 = pd.merge(df2, c_reactive_values[['person_id', 'c_reactive_final']], on = 'person_id', how = 'left')

##### creatinine feature 

Basic metabolic panel. Clinically, typically can ignore values below 1.0, and focus on continuous scale on values 1.0 and higher.

In [ ]:
creatinine_names = features_file[features_file.feature_liya == 'creatinine'].name_starr_omop
creatinine = max_date[max_date._measurement_name.isin(creatinine_names)]
creatinine_values = creatinine.pivot(index = 'person_id', 
                                       columns = ['_measurement_name'], 
                                       values = 'value_as_number').reset_index()

In [ ]:
creatinine_values.describe()

In [ ]:
units[units.labels == 'creatinine'].head()

In [ ]:
# Bring in units for each creatinine measurement
creatinine = creatinine[['person_id', 'measurement_concept_id', '_measurement_name', 'labels', 'measurement_DATETIME', 
                        'value_as_number']]

In [ ]:
creatinine = pd.merge(creatinine, measurement_features[['person_id', 'measurement_concept_id', 'measurement_DATETIME', 
                                                       'value_as_number', 'unit_source_value']], 
                     on = ['person_id', 'measurement_concept_id', 'measurement_DATETIME', 'value_as_number'], 
                     how = 'left').drop_duplicates()

In [ ]:
creatinine.unit_source_value.unique()

In [ ]:
creatinine.unit_source_value = np.where((creatinine.unit_source_value == 'MG/DL')|(creatinine.unit_source_value == 'mg/dl') 
                                       | (creatinine.unit_source_value == 'mg/dL (See scan or EMR data for detail)' )
                                       | (creatinine.unit_source_value == 'MG/DL             '), 'mg/dL', 
                                       creatinine.unit_source_value)

In [ ]:
creatinine[creatinine.unit_source_value.isnull()]

In [ ]:
creatinine.unit_source_value.unique()

In [ ]:
creatinine.measurement_concept_id.unique()

In [ ]:
# Look at nan by concept_id
creatinine[(creatinine.measurement_concept_id == 3016723) &(creatinine.unit_source_value.isna())]

Convert Creatinine [Mass/volume] in Serum or Plasma, 3016723 to mg/dL.

In [ ]:
creatinine.unit_source_value = np.where((creatinine.measurement_concept_id == 3016723) &(creatinine.unit_source_value.isna()), 
                                       'mg/dL', creatinine.unit_source_value)

In [ ]:
# Look at nan by concept_id
creatinine[(creatinine.measurement_concept_id == 4013964) &(creatinine.unit_source_value.isna())]

In [ ]:
# Convert all 'Creatinine measurement, serum' to 'mg/dL'
creatinine.unit_source_value = np.where((creatinine.measurement_concept_id==4013964) & (creatinine.unit_source_value.isnull()), 
                                                                                     'mg/dL', creatinine.unit_source_value)

In [ ]:
# Look at nan by concept_id
creatinine[(creatinine.measurement_concept_id == 4324383) &(creatinine.unit_source_value.isna())]

In [ ]:
# Convert all 'Creatinine measurement, serum' to 'mg/dL'
creatinine.unit_source_value = np.where((creatinine.measurement_concept_id==4324383) & (creatinine.unit_source_value.isnull()), 
                                                                                     'mg/dL', creatinine.unit_source_value)

In [ ]:
# If value less than 10, make mg/dL
creatinine.unit_source_value = np.where((creatinine.value_as_number < 10) & (creatinine.unit_source_value.isnull()), 
                                                                                     'mg/dL', creatinine.unit_source_value)

In [ ]:
creatinine.head()

In [ ]:
creatinine.unit_source_value.unique()

In [ ]:
creatinine.unit_source_value = np.where((creatinine.unit_source_value == 'mg/Day') | (creatinine.unit_source_value
                            == 'mg/d'), 'mg/24hr', creatinine.unit_source_value)

In [ ]:
# Remove Corrected creatinine measurement
# Concept ID: 44790124
creatinine = creatinine[creatinine.measurement_concept_id != 44790124]

In [ ]:
test = creatinine.groupby(['unit_source_value'], as_index = False).agg({
    'value_as_number': ['mean', 'min', 'max'], 
    'person_id': 'nunique'
})

In [ ]:
test

In [ ]:
creatinine.unit_source_value.unique()

In [ ]:
creatinine[creatinine.unit_source_value.isnull()]._measurement_name.unique()

In [ ]:
creatinine[creatinine._measurement_name == 'Creatinine [Mass/volume] in Urine'].unit_source_value.unique()

In [ ]:
creatinine[creatinine.unit_source_value.isna()]

In [ ]:
creatinine.unit_source_value = np.where((creatinine.unit_source_value.isnull()) & (creatinine._measurement_name == 
                                'Creatinine [Mass/volume] in Urine'), 'mg/dL', creatinine.unit_source_value)

In [ ]:
creatinine[(creatinine._measurement_name == 'Creatinine measurement') & (creatinine.unit_source_value.isnull())]

In [ ]:
creatinine.unit_source_value = np.where((creatinine.unit_source_value.isnull() )& (creatinine._measurement_name ==
                            'Creatinine measurement'), 'mg/dL', creatinine.unit_source_value)

In [ ]:
creatinine.unit_source_value.unique()

In [ ]:
# Keep all values with mg/dL as units and take max date of those
creatinine.head()

In [ ]:
creatinine_proper_units = creatinine[creatinine.unit_source_value == 'mg/dL']

In [ ]:
print(len(creatinine_proper_units))
print(len(creatinine_proper_units.person_id.unique()))

In [ ]:
creatinine_proper_units.head()

In [ ]:
creatinine_proper_units = creatinine_proper_units.sort_values(['person_id', 'measurement_DATETIME'], ascending = False).groupby(['person_id']).first().reset_index()

In [ ]:
creatinine_proper_units.head()

In [ ]:
creatinine_proper_units[creatinine_proper_units.value_as_number > 20]

In [ ]:
# To clean this remove all rows with value_as_number greater than 10
creatinine_proper_units = creatinine_proper_units[creatinine_proper_units.value_as_number < 10]

In [ ]:
print(len(creatinine_proper_units))
print(len(creatinine_proper_units.person_id.unique()))

In [ ]:
creatinine_values = creatinine_proper_units.pivot(index = 'person_id', 
                                       columns = ['_measurement_name'], 
                                       values = 'value_as_number').reset_index()

In [ ]:
creatinine_values.describe()

In [ ]:
creatinine_values['creatinine_final'] = creatinine_values['Creatinine [Mass/volume] in Serum or Plasma']

creatinine_values.creatinine_final = np.where(creatinine_values.creatinine_final.isnull(), 
                                creatinine_values['Creatinine [Mass/volume] in Urine'], creatinine_values.creatinine_final)

creatinine_values.creatinine_final = np.where(creatinine_values.creatinine_final.isnull(), 
                            creatinine_values['Creatinine measurement'], creatinine_values.creatinine_final)

creatinine_values.creatinine_final = np.where(creatinine_values.creatinine_final.isnull(), 
                            creatinine_values['Creatinine measurement, serum'], creatinine_values.creatinine_final)

creatinine_values.creatinine_final = np.where(creatinine_values.creatinine_final.isnull(), 
                            creatinine_values['Creatinine; blood'], creatinine_values.creatinine_final)

In [ ]:
creatinine_values.head()

In [ ]:
print("Number of patients with creatinine values:", len(creatinine_values.person_id.unique()))

In [ ]:
# Merge with patient df 
df2 = pd.merge(df2, creatinine_values[['person_id', 'creatinine_final']], on = 'person_id', how = 'left')

##### urine_pcr feature

Having protein in urine is red flag. Continuous variable? Need to determine value for NA.

<150 mg/24 hr normal, abnormal between 150 and 3000, worse category is greater than 3000. -1 for NA

In [ ]:
urine_pcr_names = features_file[features_file.feature_liya == 'urine_PCR'].name_starr_omop
urine_pcr = max_date[max_date._measurement_name.isin(urine_pcr_names)]
urine_pcr_values = urine_pcr.pivot(index = 'person_id', 
                                       columns = ['_measurement_name'], 
                                       values = 'value_as_number').reset_index()
urine_pcr_values.describe()

PCR in mg/g approximates Protein excretion in mg/24h.

In [ ]:
units[units.labels == 'urine_PCR']

In [ ]:
# Bring in units for each measurement
urine_pcr.drop('unit_source_value', inplace = True, axis = 1)
urine_pcr = pd.merge(urine_pcr, measurement_features[['person_id', 'measurement_concept_id', 'value_as_number', 
                                                       'measurement_DATETIME', 'unit_source_value']], 
                     on = ['person_id', 'measurement_concept_id', 'value_as_number', 'measurement_DATETIME'], 
                     how = 'left')
urine_pcr._measurement_name.unique()

In [ ]:
test = urine_pcr.groupby(['unit_source_value'], as_index = False).agg({
    'value_as_number': ['mean', 'min', 'max'], 
    'person_id': 'nunique'
})

In [ ]:
test

In [ ]:
urine_pcr['unit_source_value_final'] = urine_pcr.unit_source_value

In [ ]:
# Can assume values less than 10 are in mg/mg and those greater than 10 are in mg/g
urine_pcr.unit_source_value_final = np.where(urine_pcr.value_as_number < 10, 'mg/mg', 
                                      urine_pcr.unit_source_value_final)

urine_pcr.unit_source_value_final = np.where(urine_pcr.value_as_number > 50, 'mg/g', 
                                            urine_pcr.unit_source_value_final)

urine_pcr.unit_source_value_final = np.where(urine_pcr.unit_source_value_final == 'mg/g creat', 'mg/g', 
                                            urine_pcr.unit_source_value_final)

In [ ]:
test = urine_pcr.groupby(['unit_source_value_final'], as_index = False).agg({
    'value_as_number': ['mean', 'min', 'max'], 
    'person_id': 'nunique'
})

In [ ]:
test

In [ ]:
# Convert all to mg/g
urine_pcr.value_as_number = np.where(urine_pcr.unit_source_value_final == 'mg/g', urine_pcr.value_as_number/1000, 
                                    urine_pcr.value_as_number)
urine_pcr.value_as_number.describe()

In [ ]:
urine_pcr.unit_source_value_final = 'mg/mg'

In [ ]:
urine_pcr._measurement_name.unique()

In [ ]:
print(len(urine_pcr.person_id.unique()))

In [ ]:
urine_pcr = urine_pcr.sort_values(['person_id', 'measurement_DATETIME'], 
                                  ascending = False).groupby(['person_id']).first().reset_index()

In [ ]:
urine_pcr._measurement_name.unique()

In [ ]:
urine_pcr_values = urine_pcr.pivot(index = 'person_id', 
                                  columns = ['_measurement_name'], 
                                  values = 'value_as_number').reset_index()

In [ ]:
urine_pcr_values.describe()

In [ ]:
# Merge with patient df
urine_pcr_values.columns = ['person_id', 'urine_pcr']

In [ ]:
print("Number of patients with urine PCR values:", len(urine_pcr_values.person_id.unique()))

In [ ]:
df2 = pd.merge(df2, urine_pcr_values, on = 'person_id', how = 'left')

##### albumin feature 

If low indicates chronic disease or inflammation. Higher values in urine is worse. I think you can make this continuous after 30mg/24 hours. Categories: normal less than 30 mg/24 hours, next category (worse within the category the higher it is) 30-300mg/day, then next category, with higher getting worse, is more than 300mg per day

1: Abnormal (30-300mg/day)  
0: Normal  
-1: NA

In [ ]:
albumin_names = features_file[features_file.feature_liya == 'albumin'].name_starr_omop
albumin = max_date[max_date._measurement_name.isin(albumin_names)]
albumin_values = albumin.pivot(index = 'person_id', 
                                       columns = ['_measurement_name'], 
                                       values = 'value_as_number').reset_index()
albumin_values.describe()

In [ ]:
units[units.labels == 'albumin']

In [ ]:
albumin.drop('unit_source_value', inplace = True, axis = 1)

In [ ]:
# Merge to bring in individual units for each value for albumin
albumin = pd.merge(albumin, measurement_features[['person_id', 'measurement_concept_id', 'value_as_number', 
                                                       'measurement_DATETIME', 'unit_source_value']], 
                     on = ['person_id', 'measurement_concept_id', 'value_as_number', 'measurement_DATETIME'], 
                     how = 'left')

In [ ]:
albumin[albumin._measurement_name == 'Albumin [Mass/volume] in Serum or Plasma'].unit_source_value.unique()

In [ ]:
albumin.unit_source_value = np.where(albumin.unit_source_value == 'G/DL', 'g/dL', albumin.unit_source_value)
albumin.unit_source_value = np.where(albumin.unit_source_value == 'g/dl', 'g/dL', albumin.unit_source_value)

In [ ]:
albumin.unit_source_value.unique()

In [ ]:
albumin.unit_source_value = np.where(albumin.unit_source_value == 'MG/DL', 'g/dL', albumin.unit_source_value)

The normal range is 3.5 to 5.5 g/dL or 35-55 g/liter.

In [ ]:
# If it's null and less than 10, make units g/dL
albumin.unit_source_value = np.where((albumin.unit_source_value.isnull()) &(albumin.value_as_number < 10), 
                                    'g/dL', albumin.unit_source_value )

In [ ]:
albumin.unit_source_value.unique()

In [ ]:
albumin[(albumin._measurement_name == 'Albumin [Mass/volume] in Serum or Plasma')
        & (albumin.unit_source_value == 'mg/dL')]

albumin.value_as_number = np.where(albumin.unit_source_value == 'mg/dL', albumin.value_as_number/1000, 
                                  albumin.value_as_number)

albumin.unit_source_value = np.where(albumin.unit_source_value == 'mg/dL', 'g/dL', 
                                    albumin.unit_source_value)

albumin.unit_source_value.unique()

In [ ]:
test = albumin.groupby(['unit_source_value'], as_index = False).agg({
    'value_as_number': ['mean', 'min', 'max'], 
    'person_id': 'nunique'
})

In [ ]:
test.head()

In [ ]:
albumin[albumin.unit_source_value == '%']

In [ ]:
albumin.value_as_number = np.where(albumin.unit_source_value == '%', albumin.value_as_number/10, 
                                  albumin.value_as_number)
albumin.unit_source_value = np.where(albumin.unit_source_value == '%', 'g/dL', 
                                    albumin.unit_source_value)

In [ ]:
albumin.head()

In [ ]:
albumin = albumin.sort_values(['person_id', 'measurement_DATETIME'], 
                             ascending = False).groupby(['person_id']).first().reset_index()

In [ ]:
print("Number of patients with albumin values:", len(albumin.person_id.unique()))

In [ ]:
albumin.unit_source_value.unique()

In [ ]:
albumin._measurement_name.unique()

In [ ]:
print(len(albumin[albumin._measurement_name == 'Albumin [Mass/volume] in Urine']))

In [ ]:
albumin[albumin._measurement_name == 'Albumin [Mass/volume] in Serum or Plasma'].unit_source_value.unique()

In [ ]:
albumin[(albumin._measurement_name == 'Albumin [Mass/volume] in Urine')
       & (albumin.unit_source_value == 'ug/mL')]

In [ ]:
albumin.unit_source_value = np.where(albumin.person_id == 30526420, 'g/dL', albumin.unit_source_value)
albumin.unit_source_value = np.where(albumin.person_id == 32253578, 'g/dL', albumin.unit_source_value)
albumin.unit_source_value = np.where(albumin.person_id == 30649376, 'g/dL', albumin.unit_source_value)

In [ ]:
albumin.unit_source_value.unique()

In [ ]:
albumin[albumin.unit_source_value == 'mg/L']

In [ ]:
# Divide by 10,000?
albumin.value_as_number = np.where(albumin.unit_source_value == 'mg/L', albumin.value_as_number/10000, 
                                  albumin.value_as_number)

In [ ]:
albumin_values = albumin.pivot(index = 'person_id', 
                                       columns = ['_measurement_name'], 
                                       values = 'value_as_number').reset_index()

In [ ]:
albumin_values.describe()

In [ ]:
albumin_values = albumin.pivot(index = 'person_id', 
                                       columns = ['labels'], 
                                       values = 'value_as_number').reset_index()
albumin_values.head()

In [ ]:
# Merge with patient df
df2 = pd.merge(df2, albumin_values, on = 'person_id', how = 'left')

##### urine_ACR feature 

Indicates if you have albumin in urine. Albumin is protein normally found in blood. Normal is <30 mg/g and abnormal is anything above. Categorical variable.

1: Abnormal (>=30mg/g)  
0: Normal  
-1: NA  

In [ ]:
urine_acr_names = features_file[features_file.feature_liya == 'urine_ACR'].name_starr_omop
urine_acr = max_date[max_date._measurement_name.isin(urine_acr_names)]
urine_acr_values = urine_acr.pivot_table(index = 'person_id', 
                                       columns = ['_measurement_name'], 
                                       values = 'value_as_number').reset_index()

In [ ]:
urine_acr_values.describe()

In [ ]:
units[units.labels == 'urine_ACR']

In [ ]:
urine_acr.drop('unit_source_value', inplace = True, axis = 1)

In [ ]:
urine_acr = pd.merge(urine_acr, measurement_features[['person_id', 'measurement_concept_id', 'value_as_number', 
                                                       'measurement_DATETIME', 'unit_source_value']], 
                     on = ['person_id', 'measurement_concept_id', 'value_as_number', 'measurement_DATETIME'], 
                     how = 'left') 

In [ ]:
urine_acr.head()

In [ ]:
# Find the max date per person
urine_acr = urine_acr.sort_values(['person_id', 'measurement_DATETIME'], 
                             ascending = False).groupby(['person_id']).first().reset_index()

In [ ]:
print("Number of patients with urine_acr values:", len(urine_acr.person_id.unique()))

In [ ]:
urine_acr.unit_source_value.unique()

In [ ]:
urine_acr.unit_source_value = np.where(urine_acr.unit_source_value == 'mg/g creat', 'mg/g', urine_acr.unit_source_value)

In [ ]:
urine_acr[urine_acr.unit_source_value == 'mcg/mg creat']

mg/g is the same as mcg/mg

In [ ]:
# Update all to mg/g
urine_acr.unit_source_value = np.where(urine_acr.unit_source_value == 'mcg/mg creat', 'mg/g', 
                                      urine_acr.unit_source_value)

In [ ]:
urine_acr.describe()

In [ ]:
urine_acr_values = urine_acr.pivot(index = 'person_id', 
                                       columns = ['labels'], 
                                       values = 'value_as_number').reset_index()

In [ ]:
urine_acr_values.head()

In [ ]:
urine_acr_values.columns = ['person_id', 'urine_acr']

Can just use albumin/creatinine [ratio] in urine column. and remove mass ratio column since has n = 1. Units of mg/g.

In [ ]:
df2 = pd.merge(df2, urine_acr_values, on = 'person_id', how = 'left')

In [ ]:
df2.head()

##### gfr feature 

Kidney's filtration rate; indicates how well kidneys are filtering. >90 is normal, <60 indicates kidney disease. Categorical variable.

1: Abnormal (<60)  
0: Normal  
-1: NA

In [ ]:
gfr_names = features_file[features_file.feature_liya == 'gfr'].name_starr_omop
gfr = max_date[max_date._measurement_name.isin(gfr_names)]
gfr_values = gfr.pivot(index = 'person_id', 
                                       columns = ['_measurement_name'], 
                                       values = 'value_as_number').reset_index()

In [ ]:
gfr_values.describe()

Need to look at values.

In [ ]:
units[units.labels == 'gfr']

Looks like units of mL/min

In [ ]:
# Take last measurement for each person
# Find the max date per person
gfr = gfr.sort_values(['person_id', 'measurement_DATETIME'], 
                             ascending = False).groupby(['person_id']).first().reset_index()

In [ ]:
print(len(gfr))
print("Number of patients with grf values:", len(gfr.person_id.unique()))

In [ ]:
gfr.head()

In [ ]:
gfr._measurement_name.unique()

In [ ]:
gfr_values = gfr.pivot(index = 'person_id', 
                                       columns = ['labels'], 
                                       values = 'value_as_number').reset_index()

In [ ]:
gfr_values.describe()

In [ ]:
# Merge with patient df
df2 = pd.merge(df2, gfr_values, on = 'person_id', how = 'left')

##### systolic blood pressure 

measures the force your heart exerts on the walls of your arteries each time it beats

-1: Unknown   
0: Normal < 120   
1: Abnormal between 120 and 130  
2: Very abnormal > 130


In [ ]:
bp_names = features_file[features_file.feature_liya == 'systolic_blood_pressure'].name_starr_omop

In [ ]:
bp = max_date[max_date._measurement_name.isin(bp_names)]

In [ ]:
bp = bp.sort_values(['person_id', 'measurement_DATETIME'], 
                             ascending = False).groupby(['person_id']).first().reset_index()

In [ ]:
bp.head()

In [ ]:
bp_final = bp[['person_id', 'value_as_number']]
bp_final.columns = ['person_id', 'bp_final']

In [ ]:
# Merge with patient df
df2 = pd.merge(df2, bp_final, on = 'person_id', how = 'left')

### 8. Bin measurement features

##### Explore measurement values 

In [ ]:
def explore_values(feature_list, df):
    for col in feature_list:
        print("Column name: %s" % col)
        print("Number of NaN values: %d" % df[col].isna().sum())
        print("Number of Patients with these values: %d" % df[col].notna().sum())
        print("Imputation value (mean): %.3f" % df[col].mean())
        print()

##### anti_double_stranded_DNA_antibody

Binary: Specific type of ANA antibody present in ~30% of people with SLE. Less than 1% of healthy individuals have antibody. 

1: Present  
0: Not Present

In [ ]:
df2['DNA_antibody_binary'] = np.where(df2.anti_double_stranded_DNA_antibody.isnull(), 0, 1)

In [ ]:
explore_values(['anti_double_stranded_DNA_antibody'], df2)

##### C3 values 

Low c3 levels occur in active lupus. Categorical variable if low.   
abnormal < 86 mg/dl, normal => 86 mg/dl, unknown NA

1: Abnormal < 86 mg/dL   
0: Normal >= 86 mg/dL   
-1: Unknown NA

In [ ]:
df2['c3_binned'] = np.where(df2['c3_final'] < 86, 1,
                                  np.where(df2['c3_final']>= 86, 
                                           0, -1))

In [ ]:
explore_values(['c3_final', 'c4_final'], df2)

##### C4 values 

Low c4 levels occur in active lupus. Categorical variable if low.   
abnormal < 20 mg/dl, normal => 20mg/dl, unknown NA

1: Abnormal < 20 mg/dl    
0: Normal => 20 mg/dL  
-1: Unknown if NA

In [ ]:
df2['c4_binned'] = np.where(df2['c4_final'] < 20, 1,
                                  np.where(df2['c4_final']>= 86, 
                                           0, -1))

##### erythrocyte_sedimentation_rate feature

Lab test marker for inflammation. Faster than normal rate may indicate a systemic disease. ESR is 0–14 mm/h for males under 50 years old, 0–19 mm/h for females under 50 years old, 0–19 for males between 50 and 80 years old, and 0–29 mm/h for females between 50 and 80 years old. Categorical variable.

1: Abnormal (see above)  
0: Normal  
-1: NA

In [ ]:
explore_values(['esr_final'], df2)

In [ ]:
df2.head()

In [ ]:
df2['esr_binned'] = np.where((df2.gender == 1) & (df2.esr_final >= 29) & (df2.age_at_diagnosis>50), 
                            0, 1)

In [ ]:
df2.esr_binned = np.where((df2.gender == 1) & (df2.esr_final >=19) & (df2.age_at_diagnosis <= 50), 
                         0, df2.esr_binned)

In [ ]:
df2.esr_binned = np.where((df2.gender == 2) & (df2.esr_final >=19) & (df2.age_at_diagnosis > 50), 
                         0, df2.esr_binned)

In [ ]:
df2.esr_binned = np.where((df2.gender == 2) & (df2.esr_final >= 14) & (df2.age_at_diagnosis <= 50), 
                         0, df2.esr_binned)

In [ ]:
sum(df2.esr_binned == 0)

In [ ]:
sum(df2.esr_binned == 1)

In [ ]:
sum(df2.esr_final.isnull())

In [ ]:
df2.esr_binned = np.where(df2.esr_final.isnull(), -1, df2.esr_binned)

In [ ]:
df2.esr_binned.unique()

##### c_reactive feature 

Lab test marker for inflammation. abnormal if >= 0.5 mg/dl or 5mg/L, normal if below, NA if unknown. Categorical variable.

1: Abnormal (>= 0.5mg/dl or 5mg/L)  
0: Normal  
-1: NA

In [ ]:
explore_values(['c_reactive_final'], df2)

In [ ]:
c_reactive_values.head()

In [ ]:
df2['c_reactive_binned'] = np.where(df2['c_reactive_final'] > 0.5, 1,
                                    np.where(df2.c_reactive_final <= 0.5, 0, -1))

##### creatinine feature 

Basic metabolic panel. Can ignore values below 1.0, continuous?
Categorical: abnormal 1 if greater than 1.0, normal if less, and -1 if NA

In [ ]:
explore_values(['creatinine_final'], df2)

In [ ]:
df2['creatinine_binned'] = np.where(df2.creatinine_final.isna(), -1,
                            np.where(df2.creatinine_final <= 1.0, 0, 1) )

In [ ]:
df2.creatinine_binned.unique()

##### urine_pcr feature

Having protein in urine is red flag. I think this can be continuous when higher than 150mg/24 hr. normal<150mg per 24 hour period (makes no difference if they are below that number). Next category (higher in the category is worse) 150-3000, then next category is 3000+ (higher within that category is worse). I think you can just make this continuous after 150mg.


I think you can make this continuous after 0.2 mg/mg. less than 0.2 mg/mg is normal (makes no difference if they are below that number), then continuous measure within 0.2-3 for the next highest risk (the higher within the category, the worse), then >3 as the next category of even more risk, with higher values being worse.



2: Worse abnormal (>3)  
1: Abnormal (0.2-3)  
0: Normal (<0.2)  
-1: Unknown if NA  

In [ ]:
explore_values(['urine_pcr'], df2)

In [ ]:
urine_pcr_values.describe()

In [ ]:
df2['urine_pcr_binned'] = np.where(df2.urine_pcr >3.0, 2, np.where(df2.urine_pcr > 0.2, 1, 
                        np.where(df2.urine_pcr <= 0.2, 0, -1)))

In [ ]:
df2.urine_pcr_binned.unique()

##### albumin feature 

If low indicates chronic disease or inflammation. The normal range is 3.5 to 5.5 g/dL or 35-55 g/liter. https://loinc.org/1751-7/. Categorical variable

1: Abnormal   
0: Normal  
-1: NA

In [ ]:
explore_values(['albumin'], df2)

In [ ]:
albumin_values.describe()

In [ ]:
df2['albumin_binned'] = np.where((df2.albumin >= 3.5) & (df2.albumin <= 5.5), 
                                 0, np.where(df2.albumin.isnull(), -1, 1))

In [ ]:
df2.albumin_binned.unique()

##### urine_ACR feature 

Indicates if you have albumin in urine. Albumin is protein normally found in blood. Categories: normal less than 30mg/g, 30-300 next worse, 300+ worse. I think you can just make this a continuous variable after the 30mg/g mark. This applies to the urine albumin measurements below as well

2: Worse abnormal (300+)  
1: Abnormal (>=30mg/g)  
0: Normal  
-1: NA  


In [ ]:
explore_values(['urine_acr'], df2)

In [ ]:
df2['urine_acr_binned'] = np.where(df2.urine_acr <= 30,0, 
                        np.where(df2.urine_acr.isna(),-1, 
                        np.where(df2.urine_acr > 300, 2, 1)))

In [ ]:
df2.urine_acr_binned.unique()

##### gfr feature 

Kidney's filtration rate; indicates how well kidneys are filtering. < 60 abnormal

1: Abnormal (<60)  
0: Normal  
-1: NA

In [ ]:
explore_values(['gfr'], df2)

In [ ]:
df2['gfr_binned'] = np.where(df2.gfr <60, 1, np.where(df2.gfr.isnull(), -1, 0))

In [ ]:
df2.head()

In [ ]:
df2.gfr_binned.unique()

##### systolic blood pressure 

measures the force your heart exerts on the walls of your arteries each time it beats

-1: Unknown   
0: Normal < 120   
1: Abnormal between 120 and 130  
2: Very abnormal > 130


In [ ]:
explore_values(['bp_final'], df2)

In [ ]:
df2['bp_binned'] = np.where(df2.bp_final <= 120, 0, np.where(df2.bp_final.isnull(), -1, 
                np.where(df2.bp_final > 130, 2, 1)))

In [ ]:
df2.bp_binned.unique()

### 10. NLP features

Bring in Note_nlp features to indicate presence of condition.

In [ ]:
# # Condition occurrence table
# nlp = pd.read_csv('/home/kbechler/notebooks/data/note_nlp.tsv', sep='\t')

In [ ]:
# print(len(nlp))
# nlp.head()

In [ ]:
# Subset to confirm only bringing in conditions prior to cohort start date
# Using less than becuase the date includes all medical records of that diagnosis
# nlp = pd.merge(nlp, patient_df[['person_id', 'cohort_start_date']], on = 'person_id', how = 'left')

In [ ]:
# print(len(nlp))

In [ ]:
# # Subset cond_occ df
# nlp = nlp[nlp.condition_start_DATE < nlp.cohort_start_date]

In [ ]:
# print(len(nlp))

In [ ]:
# # Subset to appropriate columns
# features = ['person_id', '_note_nlp_name', 'note_nlp_concept_id']
# nlp_features = nlp[features]
# nlp_features['count'] = 1

In [ ]:
# # Bring in condition features from features file
# nlp_features = list(features_file[features_file.table == 'note_nlp'].feature_liya.unique())
# print(nlp_features)

In [ ]:
# nlp_features['labels']  = np.nan

In [ ]:
# feature_concepts = {}
# def get_concept_ids(feature_list):
#     for feature in feature_list:
#         concept_ids = features_file[features_file.feature_liya == feature].concept_id
#         feature_concepts[feature] = concept_ids
#         nlp_features['labels'] = np.where(nlp_features.condition_concept_id.isin(concept_ids), feature,
#                                               nlp_features.labels)

In [ ]:
# get_concept_ids(nlp_features)

In [ ]:
# nlp_features.head()

In [ ]:
# # Subset to patients in cohort
# cond_occ_features = cond_occ_features[cond_occ_features.person_id.isin(exp_df.subject_id)]

In [ ]:
# cond_occ_features.person_id.nunique()

In [ ]:
# # Transform from long to wide dataframe and count condition concept ids
# # Fill blank values with 0
# cond_occ_features_wide = cond_occ_features.pivot_table(index = 'person_id', 
#                                                       columns = ['labels'], 
#                                                       values = 'count',
#                                                       aggfunc = 'any',
#                                                       fill_value = 0).astype(int).reset_index()

In [ ]:
# cond_occ_features_wide.describe()

In [ ]:
# # Drop nan column
# cond_occ_features_wide.drop('nan', axis = 1, inplace = True)

### 11. Explore features 

##### Plot distributions of features 

In [ ]:
df2.head()

In [ ]:
df3 = df2

In [ ]:
plot_type = {'count': ['gender', 'anemia', 'proteinuria', 'synovitis','diabetes', 'hypercholesterolemia', 
                       'hypertension','azathioprine', 'cyclophosphamide', 
                       'dexamethasone', 'hydroxychloroquine', 'methylprednisolone', 'mofetil', 
                      'prednisone', 'rituximab', 'tacrolimus', 'anti_double_stranded_DNA_antibody', 
                      'race_final', 'ethnicity_final'], 
            'binned': ['c3', 'c4', 'esr', 'c_reactive', 'creatinine' 'urine_pcr', 'albumin', 
                      'urine_acr', 'gfr', 'bp'], 
            'violin': ['age_at_diagnosis']} 
            #'none': ['race_concept_id', 'ethnicity_concept_id']}
fig_size = (20, 8)

In [ ]:
def plot_feature_distributions(plot_type, df3):
    
    for col in list(df3.columns):
#         if col in plot_type['none']:
#             continue
        
        title = "Distribution of %s column values" % col
        if col in plot_type['count']:
            print_value_counts(df3, col)
            plt.figure(figsize = fig_size)
            sns.countplot(data = df3, x = col).set_title(title)
        elif col in plot_type['violin']:
            print_value_counts(df3, col)
            plt.figure(figsize = fig_size)
            sns.violinplot(data = df3, x = col).set_title(title)
        elif col in plot_type['binned']:
            bin_col = col + "_binned"
            print_value_counts(df3, bin_col)
            fig,axes = plt.subplots(1,2,figsize = fig_size)
            fig.suptitle(title)
            axes[0].set_title("Original Data")
            sns.violinplot(ax = axes[0], data = df3, x = col)
            axes[1].set_title("Binned Data")
            sns.violinplot(ax = axes[1], data = df3, x = bin_col)
        else:
            continue

In [ ]:
def print_value_counts(df3, col):
    print("Counts table for column %s" % col)
    print(df2[col].value_counts().sort_index())
    print()

In [ ]:
plot_feature_distributions(plot_type, df3)

### 11. Finalize dataset 

In [ ]:
df3.head()

In [ ]:
# Drop irrelevant/duplicate features
df3.drop(['cohort_definition_id', 'birth_date', 
          'albumin', 'c3_final', 'c4_final', 'c_reactive_final', 'creatinine_final', 'esr_final', 
         'gfr', 'urine_acr', 'urine_pcr', 'anti_double_stranded_DNA_antibody', 'bp_final',
         'cohort_end_date', 'cohort_start_date'], axis = 1, inplace = True)

In [ ]:
df3.head()

In [ ]:
# Bring in labels
labels = pd.read_csv('patient_df_labels_2022.csv')

In [ ]:
df3 = pd.merge(df3, labels[['subject_id', 'ln_5years']], left_on = 'person_id', right_on = 'subject_id', how = 'left')

In [ ]:
df3.head()

### Additional QA

In [ ]:
print("Number of patients less than 14:", sum(df3.age_at_diagnosis <14))
print("Number of patients greater than 14:", sum(df3.age_at_diagnosis >=14))

In [ ]:
# How many of those patient have outcome
print("Number of patients less than 14 with LN:", sum((df3.age_at_diagnosis <14) &(df3.ln_5years ==1)))
print("Number of patients greater than 14 with LN:", sum((df3.age_at_diagnosis >=14) &(df3.ln_5years ==1)))

### 12. Create test train split 

In [ ]:
# Remove held out test set
test_set = pd.read_csv('test_set.csv')

In [ ]:
held_out_patients = list(test_set.patient_id.unique())

In [ ]:
print(len(held_out_patients))

##### Convert to one hot encoding 

In [ ]:
df3.head()

In [ ]:
df3.gender.unique()

In [ ]:
df4 = df3

In [ ]:
# df4[['gender']] = df4[['gender']].replace([1, 2], ['female', 'male'])

In [ ]:
df4.drop(['race_binned', 'ethnicity_binned'], axis = 1, inplace = True)

In [ ]:
# df4[['anemia', 'proteinuria', 'synovitis', 'azathioprine','cyclophosphamide', 'dexamethasone', 'hydroxychloroquine',
#     'methylprednisolone', 'mycophenolate mofetil', 'prednisone', 'rituximab', 'tacrolimus', 
#      'DNA_antibody_binary']] = df4[['anemia', 'proteinuria', 'synovitis', 'azathioprine','cyclophosphamide', 
#     'dexamethasone', 'hydroxychloroquine','methylprednisolone', 'mycophenolate mofetil', 'prednisone', 'rituximab', 
#             'tacrolimus', 'DNA_antibody_binary']].replace([0,1], ['not_present', 'present'])

df4[['c3_binned', 'c4_binned', 'esr_binned', 'c_reactive_binned', 'creatinine_binned', 'albumin_binned', 
     'gfr_binned', 'bp_binned']] = df4[['c3_binned', 'c4_binned', 'esr_binned', 'c_reactive_binned', 'creatinine_binned', 
         'albumin_binned', 'gfr_binned', 'bp_binned']].replace([-1, 0, 1], ['unknown', 'normal', 'abnormal'])

df4[['urine_pcr_binned', 'urine_acr_binned']] = df4[['urine_pcr_binned', 'urine_acr_binned']].replace([-1, 0, 1, 2], 
                    ['unknown', 'normal', 'abnormal', 'v_abnormal'])

In [ ]:
df4.head()

In [ ]:
df4.describe()

In [ ]:
df4.drop(['person_id'], axis = 1, inplace = True)

In [ ]:
df4.head()

In [ ]:
# Run one hot encode
df_one_hot = df4
cat_columns = ['race_final', 'ethnicity_final','c3_binned', 'c4_binned', 'esr_binned',
       'c_reactive_binned', 'creatinine_binned', 'urine_pcr_binned',
       'albumin_binned', 'urine_acr_binned', 'gfr_binned', 'bp_binned']
for col in cat_columns:
    col_one_hot = pd.get_dummies(df4[col], prefix = col)
    df_one_hot = pd.concat((df_one_hot, col_one_hot), axis = 1).drop(col, axis = 1)

In [ ]:
df_one_hot.head()

In [ ]:
df_one_hot.isna().sum().sum()

In [ ]:
df_final = df_one_hot[~df_one_hot.subject_id.isin(held_out_patients)]

In [ ]:
print(len(df_final))

In [ ]:
df_final.drop(['subject_id'], axis = 1, inplace = True)

In [ ]:
print(len(df_one_hot))
print(df_one_hot.head())
print(df_one_hot.shape)

##### Split into train test split 

In [ ]:
# Set random seed and validation and train percentages
val = 0.25
train = 0.75
seed = 123

In [ ]:
y = df_final['ln_5years']

In [ ]:
X = df_final.loc[:, df_final.columns != 'ln_5years']

In [ ]:
X.head()

In [ ]:
# Split into train test set
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify = y, test_size = val, 
                                                          random_state = seed)

In [ ]:
print(X.shape)
print(y.shape)
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

### 13. Model Development and Validation

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

In [ ]:
all_results = {}
all_models = {}
k = 5
# Metrics to score
metric = ['accuracy', 'balanced_accuracy', 'roc_auc', 'average_precision', 'f1', 'precision', 'recall']

In [ ]:
models = [("Logistic Regression Model", LogisticRegression()),
          ("Logistic Regression CV Model", LogisticRegressionCV()), 
          ("Gradient Boosting Model", GradientBoostingClassifier()),
          ("Random Forest Model",  RandomForestClassifier())]
results = []

for name, model in models:
    kfold = model_selection.KFold(n_splits=k, shuffle=True, random_state=seed)
    scores = model_selection.cross_validate(model, X_train, y_train, scoring=metric, cv=kfold)
    results.append((name, scores))

fig_size = (10, 5)
for name, scores in results:
    scores_df = pd.DataFrame.from_dict(scores).drop(columns=["fit_time", "score_time"])
    scores_df.columns = [col[5:] for col in scores_df.columns]
    scores_df = pd.melt(scores_df, var_name="Metric", value_name="Score")
    plt.figure(figsize=fig_size)
    ax = sns.swarmplot(data=scores_df, x="Metric", y="Score")
    ax.set_title(name)
    plt.show()
    plt.savefig('/Users/katelyn/Documents/Research/plots2/models_plots.png')
    print("\n\n")

In [ ]:
results

In [ ]:
def validation_results(clf, X_val, y_val, name, results):
    y_pred = clf.predict(X_val)
    y_score = clf.predict_proba(X_val)[:, 1]
    
    acc = metrics.accuracy_score(y_val, y_pred)
    bal_acc = metrics.balanced_accuracy_score(y_val, y_pred)
    roc_auc = metrics.roc_auc_score(y_val, y_score)
    avg_prec = metrics.average_precision_score(y_val, y_score)
    f1 = metrics.f1_score(y_val, y_pred)
    precision = metrics.precision_score(y_val, y_pred)
    recall = metrics.recall_score(y_val, y_pred)     
    results[name] = {"accuracy": acc,
                     "balanced_accuracy": bal_acc,
                     "roc_auc": roc_auc,
                     "average_precision": avg_prec,
                     "f1": f1,
                     "precision": precision,
                     "recall": recall}
    
    return results

In [ ]:
solvers = ["newton-cg", "lbfgs", "liblinear", "sag", "saga"]
penalties = ["l1"]
C_values = [100, 10, 1.0, 0.1, 0.01] # smaller values = stronger regularization

i = 0
results = {}
models = []
for solver, penalty, C in it.product(solvers, penalties, C_values):
    if solver in ["newton-cg", "sag", "lbfgs"] and penalty != "l2":
        # skip parameter configurations not supported by scikit-learn
        continue
        
    print("Model #{i} Hyperparameters => Solver: {s}, Penalty: {p}, C: {c}".format(i=i, s=solver, p=penalty, c=C))
    if solver in ["sag", "saga", "liblinear"]:
        # these solvers first shuffle the data, so include random seed
        model = LogisticRegression(solver=solver, penalty=penalty, random_state=seed)
    else:
        model = LogisticRegression(solver=solver, penalty=penalty, random_state = seed)
    model.fit(X_train, y_train)
    models.append(model)
    results = validation_results(model, X_val, y_val, "Model #%d" % i, results)
    i += 1

In [ ]:
solvers = ["lbfgs"]
penalties = ["l2"]

i = 10
for solver, penalty in it.product(solvers, penalties):
    if solver in ["newton-cg", "sag", "lbfgs"] and penalty != "l2":
        # skip parameter configurations not supported by scikit-learn
        continue
        
    print("Model #{i} Hyperparameters => Solver: {s}, Penalty: {p},".format(i=i, s=solver, p=penalty))
    if solver in ["sag", "saga", "liblinear"]:
        # these solvers first shuffle the data, so include random seed
        model = LogisticRegressionCV(solver=solver, cv = 5, penalty=penalty, random_state=seed)
    else:
        model = LogisticRegressionCV(solver=solver, cv = 5, penalty=penalty, random_state = seed)
    model.fit(X_train, y_train)
    models.append(model)
    results = validation_results(model, X_val, y_val, "Model #%d" % i, results)
    i += 1

In [ ]:
results

In [ ]:
def plot_validation_results(results, model_name, fig_size=(10, 8)):
    results_df = pd.DataFrame()
    for name, result in results.items():
        results_df = results_df.append(result, ignore_index=True)

    results_df = pd.melt(results_df, var_name="Metric", value_name="Score")
    results_df["Name"] = list(results.keys()) * len(metric)

    ncol = (len(results.keys()) // 20) + 1
    plt.figure(figsize=fig_size)
    ax = sns.swarmplot(data=results_df, x="Metric", y="Score", hue="Name")
    ax.set_title("Performance of%s Models with Varied Hyperparameters" % model_name)
    plt.legend(bbox_to_anchor=(1.5, 1), loc="upper right", ncol=ncol)
    plt.savefig('/Users/katelyn/Documents/Research/plots2/'+ str(model_name)+ '_validation_results.png')
    plt.show()
    
    
    print("\n\n")

In [ ]:
plot_validation_results(results, " Logistic Regression")

In [ ]:
def filter_validation_results(results, metric_name, threshold):
    filtered_models = []
    for name, result in results.items():
        if result[metric_name] > threshold:
            print(name)
            filtered_models.append(name)
            
    return filtered_models

def find_models(filtered_results):
    for model_name in filtered_results:
        print(model_name)
        print(results[model_name])
        print()

In [ ]:
f = filter_validation_results(results, "roc_auc", 0.73)

In [ ]:
r = filter_validation_results(results, "f1", 0.06)

In [ ]:
find_models(f)

In [ ]:
def plot_roc_pr_curves(name, X_val, y_val, clf, fig_size=(10, 8)):
    fig, ax = plt.subplots(1,1,figsize=fig_size)
    roc_disp = metrics.plot_roc_curve(clf, X_val, y_val, ax=ax)
    ax.set_xlabel("False Positive Rate", fontsize = 18)
    ax.set_ylabel("True Positive Rate", fontsize = 18)
    ax.set_title("Receiver Operating Characteristic (ROC) Curve", fontsize = 18)
    plt.savefig('/Users/katelyn/Documents/Research/plots2/' + str(name) + '_val_roc.png')

    plt.show()

    # PR Curve and AP (average precision)
    fig, ax = plt.subplots(1,1,figsize=fig_size)
    pr_disp = metrics.plot_precision_recall_curve(clf, X_val, y_val, ax=ax)
    ax.set_xlabel("Recall", fontsize = 18)
    ax.set_ylabel("Precision", fontsize = 18)
    ax.set_title("Precision-Recall (PR) Curve", fontsize = 18)
    plt.savefig('/Users/katelyn/Documents/Research/plots2' + str(name) + '_val_pr.png')
    plt.show()

In [ ]:
cv_model = 10
lr_cv_model = models[cv_model]
plot_roc_pr_curves("Logistic Regression CV", X_val, y_val, lr_cv_model)
all_results = validation_results(lr_cv_model, X_val, y_val, "Logistic Regression CV", all_results)
all_models['Logistic Regression CV'] = lr_cv_model

In [ ]:
best_model_num = 4
lr_best_model = models[best_model_num]
plot_roc_pr_curves("Logistic Regression L1", X_val, y_val, lr_best_model)
all_results = validation_results(lr_best_model, X_val, y_val, "Logistic Regression L1", all_results)
all_models["Logistic Regression L1"] = lr_best_model

##### Ensemble Models: Gradient Boosting and Random Forest

In [ ]:
# Baseline model
gb_baseline = GradientBoostingClassifier(learning_rate = 0.1, n_estimators = 100, max_depth = 3, min_samples_split = 2, 
                                     min_samples_leaf = 1, subsample = 1, max_features = "sqrt", random_state = seed)
gb_baseline.fit(X_train, y_train)

all_results = validation_results(gb_baseline, X_val, y_val, "Gradient Boosting Basline", all_results)
all_models["Gradient Boosting Baseline"] = gb_baseline

In [ ]:
param_1 = {'learning_rate': [0.001, 0.005, 0.01, 0.05, 0.1, 0.15, .25, .5, .75, 1]}
param_2 = {'learning_rate':  [0.001, 0.005, 0.01, 0.05, 0.1, 0.15, .25, .5, .75, 1], 'n_estimators': [100, 250, 500, 750, 1000, 1250, 1500, 1750]}
param_3 = {'learning_rate': [0.001, 0.005, 0.01, 0.05, 0.1, 0.15, .25, .5, .75, 1], 'n_estimators': [100, 250, 500, 750, 1000, 1250, 1500, 1750], 'max_depth': [2, 3, 4]}
param_4 = {'learning_rate': [0.001, 0.005, 0.01, 0.05, 0.1, 0.15, .25, .5, .75, 1], 
              'n_estimators': [100, 250, 500, 750, 1000, 1250, 1500, 1750], 
             'max_depth': [2, 3, 4], 
             'max_features': [3, 4, 5]}

In [ ]:
# GBM with param_1
tuning = model_selection.GridSearchCV(estimator = GradientBoostingClassifier(subsample = 1, max_features= 'sqrt',
                                                                             n_estimators = 100,
                                                             max_depth = 3, min_samples_split = 2, min_samples_leaf = 1,
                                                             random_state = seed), 
                     param_grid = param_1, scoring = 'accuracy', n_jobs = 4, cv = 5)
tuning.fit(X_train, y_train)
tuning.cv_results_, tuning.best_params_, tuning.best_score_

In [ ]:
# GBM with param_2
tuning = model_selection.GridSearchCV(estimator = GradientBoostingClassifier(subsample = 1, max_features= 'sqrt',
                                                             max_depth = 3, min_samples_split = 2, min_samples_leaf = 1,
                                                             random_state = seed), 
                     param_grid = param_2, scoring = 'accuracy', n_jobs = 4, cv = 5)
tuning.fit(X_train, y_train)
tuning.cv_results_, tuning.best_params_, tuning.best_score_

In [ ]:
# GBM with param_3
tuning = model_selection.GridSearchCV(estimator = GradientBoostingClassifier(subsample = 1, max_features= 'sqrt',
                                                             min_samples_split = 2, min_samples_leaf = 1,
                                                             random_state = seed), 
                     param_grid = param_3, scoring = 'accuracy', n_jobs = 4, cv = 5)
tuning.fit(X_train, y_train)
tuning.cv_results_, tuning.best_params_, tuning.best_score_

In [ ]:
# GBM with param_4
tuning = model_selection.GridSearchCV(estimator = GradientBoostingClassifier(subsample = 1, 
                                                             min_samples_split = 2, min_samples_leaf = 1,
                                                             random_state = seed), 
                     param_grid = param_4, scoring = 'accuracy', n_jobs = 4, cv = 5)

tuning.fit(X_train, y_train)
tuning.cv_results_, tuning.best_params_, tuning.best_score_

In [ ]:
gb_model_param1 = GradientBoostingClassifier(learning_rate = 0.001, random_state = seed)
gb_model_param1.fit(X_train, y_train)
predictions = gb_model_param1.predict(X_val)
print(metrics.classification_report(y_val, predictions))

In [ ]:
gb_model_param2 = GradientBoostingClassifier(learning_rate = 0.005,n_estimators = 500, random_state = seed)
gb_model_param2.fit(X_train, y_train)
predictions = gb_model_param2.predict(X_val)
print(metrics.classification_report(y_val, predictions))

In [ ]:
gb_model_param3 = GradientBoostingClassifier(learning_rate = 0.001, n_estimators = 1500, max_depth = 4, random_state = seed)
gb_model_param3.fit(X_train, y_train)
predictions = gb_model_param3.predict(X_val)
print(metrics.classification_report(y_val, predictions))

In [ ]:
gb_model_param4 = GradientBoostingClassifier(learning_rate = 0.01, max_depth = 3, max_features = 5,
                                             n_estimators = 250, random_state = seed)
gb_model_param4.fit(X_train, y_train)
predictions = gb_model_param4.predict(X_val)
print(metrics.classification_report(y_val, predictions))

In [ ]:
gb_best_model = gb_model_param3
all_results = validation_results(gb_best_model, X_val, y_val, "GB best model", all_results)
all_models["GB best model"] = gb_best_model

In [ ]:
plot_roc_pr_curves("Gradient Boosting Baseline", X_val, y_val, gb_baseline)

In [ ]:
rf = RandomForestClassifier(random_state = seed)

# Look at parameters in rf 
print('Parameters currently in use:\n')
pprint(rf.get_params())

In [ ]:
rf_baseline = RandomForestClassifier(random_state = seed)
rf_baseline.fit(X_train, y_train)

all_results = validation_results(rf_baseline, X_val, y_val, "Random Forest Baseline", all_results)
all_models["Random Forest Baseline"] = rf_baseline

In [ ]:
# Random search grid
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree 
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node 
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

In [ ]:
# Create random grid
random_grid = {'n_estimators': n_estimators, 
              'max_features': max_features, 
              'max_depth': max_depth, 
              'min_samples_split': min_samples_split, 
              'min_samples_leaf': min_samples_leaf, 
              'bootstrap': bootstrap}

pprint(random_grid)

In [ ]:
# Random search training
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier(random_state = seed)
# Random search of parameters, using 3 fold cross validation
# Search across 100 different combinations, and use all available cores
rf_random = model_selection.RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5, random_state = seed, 
                              n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)
rf_random_model = rf_random.best_estimator_

all_results = validation_results(rf_random_model, X_val, y_val, "Random Forest (Best Random Search)", all_results)
all_models["Random Forest (Best Random Search)"] = rf_random_model

# View best parameters
rf_random.best_params_

In [ ]:
plot_roc_pr_curves("Random Forest (Best Random Search)", X_val, y_val, rf_random_model)

In [ ]:
param_grid = {
    'bootstrap':[True],
    'max_depth': [9, 10, 11], 
    'max_features': [3, 4, 5, 6],
    'min_samples_leaf': [1, 2, 3, 4],
    'min_samples_split': [1, 2, 3, 4], 
    'n_estimators': [400, 500, 600, 700, 800, 900]
}

In [ ]:
# Create a base model
rf = RandomForestClassifier(random_state = seed)

# Instantiate the grid search model
grid_search = model_selection.GridSearchCV(estimator = rf, param_grid = param_grid, cv = 5, n_jobs = -1, verbose = 2)

In [ ]:
# Fit grid to search data
grid_search.fit(X_train, y_train)

# Best model
rf_grid_model = grid_search.best_estimator_

all_results = validation_results(rf_grid_model, X_val, y_val, "RF best model (Grid Search)", all_results)
all_models["Rf best model (Grid Search)"] = rf_grid_model

grid_search.best_params_

In [ ]:
all_models

In [ ]:
models_df = pd.DataFrame.from_dict(all_results, orient='index', columns = ['accuracy', 'balanced_accuracy',
                                                                               'roc_auc', 'average_precision', 
                                                                               'f1', 'precision', 'recall'])


In [ ]:
plot_validation_results(all_results, "All Models")

In [ ]:
plot_roc_pr_curves("Logistic Regression CV", X_val, y_val, lr_cv_model)

### 14. Addressing Class imbalance

In [ ]:
os_models = {}
os_results = {}

https://www.analyticsvidhya.com/blog/2020/07/10-techniques-to-deal-with-class-imbalance-in-machine-learning/

In [ ]:
#from sklearn.svm import SVC
#svc_model = SVC(class_weight='balanced', probability=True)
#svc_model.fit(X_train, y_train)
# svc_predict = svc_model.predict(X_val)# check performance
# print('ROCAUC score:',metrics.roc_auc_score(y_val, svc_predict))
# print('Accuracy score:',metrics.accuracy_score(y_val, svc_predict))
# print('F1 score:',metrics.f1_score(y_val, svc_predict))

##### Importance Weighting, Manual Weighting

##### Logistic Regression L1 Penalization 

In [ ]:
lr_pen_os = LogisticRegressionCV(solver='liblinear', random_state = seed, cv = 5, penalty = 'l1')

#Setting the range for class weights
weights = np.linspace(0.0,0.99,200)

#Creating a dictionary grid for grid search
param_grid = {'class_weight': [{0:x, 1:1.0-x} for x in weights]}

#Fitting grid search to the train data with 5 folds
gridsearch = GridSearchCV(estimator= lr_pen_os, 
                          param_grid= param_grid,
                          cv=StratifiedKFold(), 
                          n_jobs=-1, 
                          scoring='f1', 
                          verbose=2).fit(X_train, y_train)

#Ploting the score for different values of weight
sns.set_style('whitegrid')
plt.figure(figsize=(12,8))
weigh_data = pd.DataFrame({ 'score': gridsearch.cv_results_['mean_test_score'], 'weight': (1- weights)})
sns.lineplot(weigh_data['weight'], weigh_data['score'])
plt.xlabel('Weight for class 1')
plt.ylabel('F1 score')
plt.xticks([round(i/10,1) for i in range(0,11,1)])
plt.title('Scoring for different class weights', fontsize=24)

In [ ]:
# Get weight of max F1 score
value = max(weigh_data.score)
weigh_data[weigh_data.score == value]

In [ ]:
#importing and training the model

lr_pen_os = LogisticRegressionCV(solver='liblinear', penalty = 'l1', cv = 5, random_state = seed, 
                                 class_weight={0: 1-0.53, 1: 0.53})
lr_pen_os.fit(X_train, y_train)

# Predicting on the test data
pred_val = lr_pen_os.predict(X_val)

#Calculating and printing the f1 score 

print('ROCAUC score:',metrics.roc_auc_score(y_val, pred_val))
print('Accuracy score:',metrics.accuracy_score(y_val, pred_val))
print('F1 score:',metrics.f1_score(y_val, pred_val))

os_results = validation_results(lr_pen_os, X_val, y_val, "Logistic Regression L1 (Importance Weighting)",
                                os_results )
os_models['Logistic Regression L1 (Importance Weighting)'] = lr_pen_os

##### Logistic Regression CV

In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
lr_cv_os = LogisticRegressionCV(solver='lbfgs', random_state = seed, cv = 5)

#Setting the range for class weights
weights = np.linspace(0.0,0.99,200)

#Creating a dictionary grid for grid search
param_grid = {'class_weight': [{0:x, 1:1.0-x} for x in weights]}

#Fitting grid search to the train data with 5 folds
gridsearch = GridSearchCV(estimator= lr_cv_os, 
                          param_grid= param_grid,
                          cv=StratifiedKFold(), 
                          n_jobs=-1, 
                          scoring='f1', 
                          verbose=2).fit(X_train, y_train)

#Ploting the score for different values of weight
sns.set_style('whitegrid')
plt.figure(figsize=(12,8))
weigh_data = pd.DataFrame({ 'score': gridsearch.cv_results_['mean_test_score'], 'weight': (1- weights)})
sns.lineplot(weigh_data['weight'], weigh_data['score'])
plt.xlabel('Weight for class 1')
plt.ylabel('F1 score')
plt.xticks([round(i/10,1) for i in range(0,11,1)])
plt.title('Scoring for different class weights', fontsize=24)

In [ ]:
# Get weight of max F1 score
value = max(weigh_data.score)
weigh_data[weigh_data.score == value]

In [ ]:
#importing and training the model

lr_cv_os = LogisticRegressionCV(solver='lbfgs', cv = 5, random_state = seed, class_weight={0: 1-0.940302, 1: 0.940302})
lr_cv_os.fit(X_train, y_train)

# Predicting on the test data
pred_val = lr_cv_os.predict(X_val)

#Calculating and printing the f1 score 

print('ROCAUC score:',metrics.roc_auc_score(y_val, pred_val))
print('Accuracy score:',metrics.accuracy_score(y_val, pred_val))
print('F1 score:',metrics.f1_score(y_val, pred_val))

os_results = validation_results(lr_cv_os, X_val, y_val, "Logistic Regression CV (Importance Weighting)",
                                os_results )
os_models['Logistic Regression CV (Importance Weighting)'] = lr_cv_os

In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
rf_os = RandomForestClassifier(random_state = seed)

#Setting the range for class weights
weights = np.linspace(0.0,0.99,200)

#Creating a dictionary grid for grid search
param_grid = {'class_weight': [{0:x, 1:1.0-x} for x in weights]}

#Fitting grid search to the train data with 5 folds
gridsearch = GridSearchCV(estimator= rf_os, 
                          param_grid= param_grid,
                          cv=StratifiedKFold(), 
                          n_jobs=-1, 
                          scoring='f1', 
                          verbose=2).fit(X_train, y_train)

#Ploting the score for different values of weight
sns.set_style('whitegrid')
plt.figure(figsize=(12,8))
weigh_data = pd.DataFrame({ 'score': gridsearch.cv_results_['mean_test_score'], 'weight': (1- weights)})
sns.lineplot(weigh_data['weight'], weigh_data['score'])
plt.xlabel('Weight for class 1')
plt.ylabel('F1 score')
plt.xticks([round(i/10,1) for i in range(0,11,1)])
plt.title('Scoring for different class weights', fontsize=24)

In [ ]:
# Get weight of max F1 score
value = max(weigh_data.score)
weigh_data[weigh_data.score == value]

In [ ]:
#importing and training the model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
rf_os = RandomForestClassifier(random_state = seed, class_weight={0: 1-0.99, 1: 0.99})
rf_os.fit(X_train, y_train)

# Predicting on the test data
pred_val = rf_os.predict(X_val)

#Calculating and printing the f1 score 

print('ROCAUC score:',metrics.roc_auc_score(y_val, pred_val))
print('Accuracy score:',metrics.accuracy_score(y_val, pred_val))
print('F1 score:',metrics.f1_score(y_val, pred_val))

os_results = validation_results(rf_os, X_val, y_val, "Random Forest (Importance Weighting)",
                                os_results )
os_models['Random Forest (Importance Weighting)'] = rf_os

##### SMOTE for class imbalance 

https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/

In [ ]:
print(len(X_train))
print(len(y_train))
print(len(X_val))
print(len(y_val))

In [ ]:
from sklearn.datasets import make_classification
from collections import Counter

counter = Counter(y_train)
print("before:", counter)

X, y = make_classification(n_classes=2, class_sep=2,
weights=[0.1, 0.9], n_informative=3, n_redundant=1, flip_y=0,
n_features=59, n_clusters_per_class=1, n_samples=1000, random_state=10)

from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=seed, k_neighbors = 3)
X_res, y_res = sm.fit_resample(X_train, y_train)
counter = Counter(y_res)
print("after:", counter)

In [ ]:
# Generate Random Forest Model with balanced dataset using SMOTE
rf_rebalanced = RandomForestClassifier(random_state = seed)
rf_rebalanced.fit(X_res, y_res)
os_results = validation_results(rf_rebalanced, X_val, y_val, "Random Forest (SMOTE)",os_results )
os_models['Random Forest (SMOTE)'] = rf_rebalanced

In [ ]:
lr_rebalanced = LogisticRegression(random_state = seed)
lr_rebalanced.fit(X_res, y_res)
os_results = validation_results(lr_rebalanced, X_val, y_val, "Logistic Regression (SMOTE)", os_results)
os_models['Logistic Regression (SMOTE)'] = lr_rebalanced

In [ ]:
gb_rebalanced = GradientBoostingClassifier(random_state = seed)
gb_rebalanced.fit(X_res,y_res)
os_results = validation_results(gb_rebalanced, X_val, y_val, "Gradient Boosting (SMOTE)", os_results)
os_models['Gradient Boosting (SMOTE)'] = gb_rebalanced

In [ ]:
os_results

In [ ]:
X_train = X_res
y_train = y_res

##### TomekLinks 

In [ ]:
# from imblearn.under_sampling import TomekLinks

# tl = TomekLinks()
# X_resampled, y_resampled = tl.fit_resample(X_train, y_train)

# y_resampled.value_counts()

In [ ]:
# clf = LogisticRegression(random_state = seed)
# clf.fit(X_resampled, y_resampled)
# test = validation_results(clf, X_val, y_val, "TomekLinks", test)

##### Using kmeans 

In [ ]:
# from imblearn.under_sampling import ClusterCentroids

# cc = ClusterCentroids(random_state=seed)
# X_resampled, y_resampled = cc.fit_resample(X_train, y_train)

# y_resampled.value_counts()

In [ ]:
# clf = LogisticRegression(random_state = seed)
# clf.fit(X_resampled, y_resampled)
# test = validation_results(clf, X_val, y_val, "ClusterCentroids", test)

##### Use near miss

In [ ]:
# from imblearn.under_sampling import NearMiss 
# nr = NearMiss() 
# X_near, y_near= nr.fit_resample(X_train, y_train.ravel()) 
# c=Counter(y_near)

In [ ]:
# print(c)

In [ ]:
# lr = LogisticRegression()
# lr.fit(X_near, y_near)

In [ ]:
# rf = RandomForestClassifier()
# rf.fit(X_near, y_near)
# test = validation_results(rf, X_val, y_val, "RF near miss", test)

##### Tune hyperparameters 

In [ ]:
# Metrics to score
metric = ['accuracy', 'balanced_accuracy', 'roc_auc', 'average_precision', 'f1', 'precision', 'recall']
k = 5

In [ ]:
models = [("Logistic Regression Model", LogisticRegression()),
          ("Logistic Regression CV Model", LogisticRegressionCV()), 
          ("Gradient Boosting Model", GradientBoostingClassifier()),
          ("Random Forest Model",  RandomForestClassifier())]
results = []

for name, model in models:
    kfold = model_selection.KFold(n_splits=k, shuffle=True, random_state=seed)
    scores = model_selection.cross_validate(model, X_train, y_train, scoring=metric, cv=kfold)
    results.append((name, scores))

fig_size = (10, 5)
for name, scores in results:
    scores_df = pd.DataFrame.from_dict(scores).drop(columns=["fit_time", "score_time"])
    scores_df.columns = [col[5:] for col in scores_df.columns]
    scores_df = pd.melt(scores_df, var_name="Metric", value_name="Score")
    plt.figure(figsize=fig_size)
    ax = sns.swarmplot(data=scores_df, x="Metric", y="Score")
    ax.set_title(name)
    plt.show()
    plt.savefig('/Users/katelyn/Documents/Research/plots2/models_plots.png')
    print("\n\n")

In [ ]:
results

In [ ]:
solvers = ["newton-cg", "lbfgs", "liblinear", "sag", "saga"]
penalties = ["l1"]
C_values = [100, 10, 1.0, 0.1, 0.01] # smaller values = stronger regularization

results = {}
models = []
i = 0

for solver, penalty, C in it.product(solvers, penalties, C_values):
    if solver in ["newton-cg", "sag", "lbfgs"] and penalty != "l2":
        # skip parameter configurations not supported by scikit-learn
        continue
        
    print("Model #{i} Hyperparameters => Solver: {s}, Penalty: {p}, C: {c}".format(i=i, s=solver, p=penalty, c=C))
    if solver in ["sag", "saga", "liblinear"]:
        # these solvers first shuffle the data, so include random seed
        model = LogisticRegression(solver=solver, penalty=penalty, random_state=seed)
    else:
        model = LogisticRegression(solver=solver, penalty=penalty, random_state = seed)
    model.fit(X_train, y_train)
    models.append(model)
    results = validation_results(model, X_val, y_val, "Model #%d" % i, results)
    i += 1

In [ ]:
solvers = ["lbfgs"]
penalties = ["l2"]

i = 10
for solver, penalty in it.product(solvers, penalties):
    if solver in ["newton-cg", "sag", "lbfgs"] and penalty != "l2":
        # skip parameter configurations not supported by scikit-learn
        continue
        
    print("Model #{i} Hyperparameters => Solver: {s}, Penalty: {p},".format(i=i, s=solver, p=penalty))
    if solver in ["sag", "saga", "liblinear"]:
        # these solvers first shuffle the data, so include random seed
        model = LogisticRegressionCV(solver=solver, penalty=penalty, random_state=seed)
    else:
        model = LogisticRegressionCV(solver=solver, penalty=penalty, random_state = seed)
    model.fit(X_train, y_train)
    models.append(model)
    results = validation_results(model, X_val, y_val, "Model #%d" % i, results)
    i += 1

In [ ]:
plot_validation_results(results, "Logistic Regression SMOTE")

In [ ]:
f = filter_validation_results(results, "f1", 0.17)

In [ ]:
r = filter_validation_results(results, "roc_auc", 0.68)

In [ ]:
find_models(r)

In [ ]:
models

In [ ]:
best_model_num = 0
lr_smote = models[best_model_num]
plot_roc_pr_curves("Logistic Regression (SMOTE)", X_val, y_val, lr_smote)
os_results = validation_results(lr_smote, X_val, y_val, "Logistic Regression (SMOTE)", os_results)
os_models["Logistic Regression (SMOTE)"] = lr_smote

In [ ]:
best_model_num = 10
lr_cv_smote = models[best_model_num]
plot_roc_pr_curves("Logistic Regression CV (SMOTE)", X_val, y_val, lr_cv_smote)
os_results = validation_results(lr_cv_smote, X_val, y_val, "Logistic Regression CV (SMOTE)", os_results)
os_models["Logistic Regression CV (SMOTE)"] = lr_cv_smote

##### Ensemble Models: Gradient Boosting and Random Forest

In [ ]:
# Baseline model
gb_os_baseline = GradientBoostingClassifier(learning_rate = 0.1, n_estimators = 100, max_depth = 3, min_samples_split = 2, 
                                     min_samples_leaf = 1, subsample = 1, max_features = "sqrt", random_state = seed)
gb_os_baseline.fit(X_train, y_train)

os_results = validation_results(gb_os_baseline, X_val, y_val, "Gradient Boosting (SMOTE)", os_results)
os_models["Gradient Boosting (SMOTE)"] = gb_os_baseline
plot_roc_pr_curves("Gradient Boosting (SMOTE)", X_val, y_val, gb_os_baseline)

In [ ]:
param_1 = {'learning_rate': [0.001, 0.005, 0.01, 0.05, 0.1, 0.15, .25, .5, .75, 1]}
param_2 = {'learning_rate':  [0.001, 0.005, 0.01, 0.05, 0.1, 0.15, .25, .5, .75, 1], 'n_estimators': [100, 250, 500, 750, 1000, 1250, 1500, 1750]}
param_3 = {'learning_rate': [0.001, 0.005, 0.01, 0.05, 0.1, 0.15, .25, .5, .75, 1], 'n_estimators': [100, 250, 500, 750, 1000, 1250, 1500, 1750], 'max_depth': [2, 3, 4]}
param_4 = {'learning_rate': [0.001, 0.005, 0.01, 0.05, 0.1, 0.15, .25, .5, .75, 1], 
              'n_estimators': [100, 250, 500, 750, 1000, 1250, 1500, 1750], 
           
           'max_depth': [2, 3, 4], 
             'max_features': [3, 4, 5]}

In [ ]:
# GBM with param_1
tuning = model_selection.GridSearchCV(estimator = GradientBoostingClassifier(subsample = 1, max_features= 'sqrt',
                                                                             n_estimators = 100,
                                                             max_depth = 3, min_samples_split = 2, min_samples_leaf = 1,
                                                             random_state = seed), 
                     param_grid = param_1, scoring = 'accuracy', n_jobs = 4, cv = 5)
tuning.fit(X_train, y_train)
tuning.cv_results_, tuning.best_params_, tuning.best_score_

In [ ]:
# GBM with param_2
tuning = model_selection.GridSearchCV(estimator = GradientBoostingClassifier(subsample = 1, max_features= 'sqrt',
                                                             max_depth = 3, min_samples_split = 2, min_samples_leaf = 1,
                                                             random_state = seed), 
                     param_grid = param_2, scoring = 'accuracy', n_jobs = 4, cv = 5)
tuning.fit(X_train, y_train)
tuning.cv_results_, tuning.best_params_, tuning.best_score_

In [ ]:
# GBM with param_3
tuning = model_selection.GridSearchCV(estimator = GradientBoostingClassifier(subsample = 1, max_features= 'sqrt',
                                                             min_samples_split = 2, min_samples_leaf = 1,
                                                             random_state = seed), 
                     param_grid = param_3, scoring = 'accuracy', n_jobs = 4, cv = 5)
tuning.fit(X_train, y_train)
tuning.cv_results_, tuning.best_params_, tuning.best_score_

In [ ]:
# GBM with param_4
tuning = model_selection.GridSearchCV(estimator = GradientBoostingClassifier(subsample = 1, 
                                                             min_samples_split = 2, min_samples_leaf = 1,
                                                             random_state = seed), 
                     param_grid = param_4, scoring = 'accuracy', n_jobs = 4, cv = 5)

tuning.fit(X_train, y_train)
tuning.cv_results_, tuning.best_params_, tuning.best_score_

In [ ]:
gb_model_param1 = GradientBoostingClassifier(learning_rate = 0.001, random_state = seed)
gb_model_param1.fit(X_train, y_train)
predictions = gb_model_param1.predict(X_val)
print(metrics.classification_report(y_val, predictions))

In [ ]:
gb_model_param2 = GradientBoostingClassifier(learning_rate = 0.005,n_estimators = 500, random_state = seed)
gb_model_param2.fit(X_train, y_train)
predictions = gb_model_param2.predict(X_val)
print(metrics.classification_report(y_val, predictions))

In [ ]:
gb_model_param3 = GradientBoostingClassifier(learning_rate = 0.001, n_estimators = 1500, max_depth = 4, random_state = seed)
gb_model_param3.fit(X_train, y_train)
predictions = gb_model_param3.predict(X_val)
print(metrics.classification_report(y_val, predictions))

In [ ]:
gb_model_param4 = GradientBoostingClassifier(learning_rate = 0.01, max_depth = 3, max_features = 5,
                                             n_estimators = 250, random_state = seed)
gb_model_param4.fit(X_train, y_train)
predictions = gb_model_param4.predict(X_val)
print(metrics.classification_report(y_val, predictions))

In [ ]:
gb_os_best_model = gb_model_param2
os_results = validation_results(gb_os_best_model, X_val, y_val, "Gradient Boosting Best (SMOTE)", os_results)
os_models["Gradient Boosting Best (SMOTE)"] = gb_os_best_model

In [ ]:
plot_roc_pr_curves("Gradient Boosting Best (SMOTE)", X_val, y_val, gb_os_best_model)

In [ ]:
rf = RandomForestClassifier(random_state = seed)

# Look at parameters in rf 
print('Parameters currently in use:\n')
pprint(rf.get_params())

In [ ]:
rf_os_baseline = RandomForestClassifier(random_state = seed)
rf_os_baseline.fit(X_train, y_train)

os_results = validation_results(rf_os_baseline, X_val, y_val, "Random Forest (SMOTE)", os_results)
os_models["Random Forest (SMOTE)"] = rf_os_baseline
plot_roc_pr_curves("Random Forest (SMOTE)", X_val, y_val, rf_os_baseline)

In [ ]:
# Random search grid
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree 
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node 
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

In [ ]:
# Create random grid
random_grid = {'n_estimators': n_estimators, 
              'max_features': max_features, 
              'max_depth': max_depth, 
              'min_samples_split': min_samples_split, 
              'min_samples_leaf': min_samples_leaf, 
              'bootstrap': bootstrap}

pprint(random_grid)

In [ ]:
# Random search training
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier(random_state = seed)
# Random search of parameters, using 3 fold cross validation
# Search across 100 different combinations, and use all available cores
rf_random = model_selection.RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, random_state = seed, 
                              n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)
rf_os_random_model = rf_random.best_estimator_

os_results = validation_results(rf_os_random_model, X_val, y_val, "Random Forest Random (SMOTE)", os_results)
os_models["Random Forest Random (SMOTE)"] = rf_os_random_model

# View best parameters
rf_random.best_params_

In [ ]:
plot_roc_pr_curves("Random Forest Random (SMOTE)",X_val, y_val, rf_os_random_model)

In [ ]:
param_grid = {
    'bootstrap':[True],
    'max_depth': [9, 10, 11], 
    'max_features': [3, 4, 5, 6],
    'min_samples_leaf': [1, 2, 3, 4],
    'min_samples_split': [1, 2, 3, 4], 
    'n_estimators': [400, 500, 600, 700, 800, 900]
}

In [ ]:
# Create a base model
rf = RandomForestClassifier(random_state = seed)

# Instantiate the grid search model
grid_search = model_selection.GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
# Fit grid to search data
grid_search.fit(X_train, y_train)

# Best model
rf_os_grid_model = grid_search.best_estimator_

os_results = validation_results(rf_os_grid_model, X_val, y_val, "Random Forest Grid (SMOTE)", os_results)
os_models["Random Forest Grid (SMOTE)"] = rf_os_grid_model
plot_roc_pr_curves("Random Forest Grid (SMOTE)", X_val, y_val, rf_os_grid_model)

grid_search.best_params_

In [ ]:
os_results

In [ ]:
os_models_df = pd.DataFrame.from_dict(os_results, orient='index', columns = ['accuracy', 'balanced_accuracy',
                                                                               'roc_auc', 'average_precision', 
                                                                               'f1', 'precision', 'recall'])


In [ ]:
models_df

In [ ]:
plot_validation_results(os_results, "SMOTE Results")

In [ ]:
# Write ensemble models to csv
models_df.to_csv("models_df.csv")
os_models_df.to_csv("os_models_df.csv")

### 15. Model Evaluation - Held Out Test Set

In [ ]:
# Generate held out dataset
held_out_df = df_one_hot[df_one_hot.subject_id.isin(held_out_patients)]
y_heldout = held_out_df['ln_5years']
X_heldout = held_out_df.loc[:, held_out_df.columns != 'ln_5years']
X_heldout.drop([ 'subject_id'], inplace = True, axis = 1)

In [ ]:
print(len(X_heldout))
print(len(y_heldout))

In [ ]:
def test_model(name, clf, X_test, y_test, fig_size=(10,8)):
    # clf: trained classifier (i.e. after using fit function)
    y_pred = clf.predict(X_test)
    y_pred_proba = clf.predict_proba(X_test)

    # print out stats
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print("Model accuracy: %.3f\n" % accuracy)

    # precision, recall, and f1-score is usually reported for class 1 (in binary case)
    # recall of positive class (1) = sensitivity
    # recall of negative class (0) = specificity
    # precision of positive class (1) = PPV
    # precision of negative class (0) = NPV
    print(metrics.classification_report(y_test,y_pred))

    # confusion matrix
    cm_disp = metrics.plot_confusion_matrix(clf, X_test, y_test)  
    plt.savefig('/Users/katelyn/Documents/Research/plots2/' + str(name) + '_cm_test.png')
    plt.show()

    # ROC Curve and AUC
    auroc = metrics.roc_auc_score(y_test, y_pred_proba[:, 1])
    fig_roc, ax_roc = plt.subplots(1,1,figsize=fig_size)
    roc_disp = metrics.plot_roc_curve(clf, X_test, y_test, ax=ax_roc)
    ax_roc.set_xlabel("False Positive Rate", fontsize = 18)
    ax_roc.set_ylabel("True Positive Rate", fontsize = 18)
    ax_roc.set_title("Receiver Operating Characteristic (ROC) Curve", fontsize = 18)
    plt.savefig('/Users/katelyn/Documents/Research/plots2/' + str(name) + '_roc_test.png')
    plt.show()

    # PR Curve and AP (average precision)
    fig, ax_pr = plt.subplots(1,1,figsize=fig_size)
    pr_disp = metrics.plot_precision_recall_curve(clf, X_test, y_test, ax=ax_pr)
    ax_pr.set_xlabel("Recall", fontsize = 18)
    ax_pr.set_ylabel("Precision", fontsize = 18)
    ax_pr.set_title("Precision-Recall (PR) Curve", fontsize = 18)
    plt.savefig('/Users/katelyn/Documents/Research/plots2/' + str(name) + '_pr_test.png')
    plt.show()

In [ ]:
test_model("lr", lr, X_heldout, y_heldout)

In [ ]:
# Plot both validation and test roc on same plot

def plot_both(name, clf, X_val, y_val, X_test, y_test, fig_size=(10,8)):
    # clf: trained classifier (i.e. after using fit function)
    y_pred = clf.predict(X_val)
    y_pred_proba = clf.predict_proba(X_val)


    # ROC Curve and AUC
    auroc = metrics.roc_auc_score(y_val, y_pred_proba[:, 1])
    fig_roc, ax_roc = plt.subplots(1,1,figsize=fig_size)
    roc_disp = metrics.plot_roc_curve(clf, X_val, y_val, ax=ax_roc, name = "Internal Validation")
    ax_roc.set_xlabel("False Positive Rate", fontsize = 18)
    ax_roc.set_ylabel("True Positive Rate", fontsize = 18)
    ax_roc.set_title("Receiver Operating Characteristic (ROC) Curve " + str(name), fontsize = 20)
    
    y_pred_test = clf.predict(X_test)
    y_pred_proba_test = clf.predict_proba(X_test)


    auroc_test = metrics.roc_auc_score(y_test, y_pred_proba_test[:, 1])

    roc_disp = metrics.plot_roc_curve(clf, X_test, y_test, ax=ax_roc, name = "Internal Evaluation on Held-Out Test Set")
    plt.savefig('/Users/katelyn/Documents/Research/plots1/' + str(name) + '_both_roc.png')
    
    plt.show()

In [ ]:
plot_both("lr_best_model", lr_best_model, X_val, y_val, X_heldout, y_heldout)

In [ ]:
test_results = {}
test_results = validation_results(lr_best_model, X_heldout, y_heldout,"lr_best_model", test_results)

In [ ]:
test_results = validation_results(lr_cv_model, X_heldout, y_heldout,"lr_cv_model", test_results)

In [ ]:
test_results = validation_results(rf_baseline, X_heldout, y_heldout,"rf_baseline", test_results)

In [ ]:
test_results = validation_results(rf_os_random_model, X_heldout, y_heldout, "rf_os_random_model", test_results)

In [ ]:
test_results = validation_results(gb_baseline, X_heldout, y_heldout, "gb_baseline", test_results)

In [ ]:
# Cohort demographics

In [ ]:
print("Num female:", sum(outcome_yes.gender == 1))
print("% female:", sum(outcome_yes.gender == 1)/len(outcome_yes))
print("Num female outcome no:", sum(outcome_no.gender == 1))
print("% female outcome no:", sum(outcome_no.gender == 1)/len(outcome_no))

In [ ]:
2608-2367

In [ ]:
155-133

In [ ]:
outcome_yes.head()

In [ ]:
race = outcome_yes.groupby(['race_final'],as_index = False).agg({
    'subject_id': 'nunique'
})

In [ ]:
race['total'] = 155
race['percent'] = race.subject_id/race.total
race

In [ ]:
race1 = outcome_no.groupby(['race_final'],as_index = False).agg({
    'subject_id': 'nunique'
})
race1['total'] = 2608
race1['percent'] = race1.subject_id/race1.total
race1

In [ ]:
eth = outcome_no.groupby(['ethnicity_final'],as_index = False).agg({
    'subject_id': 'nunique'
})
eth['total'] = 2608
eth['percent'] = eth.subject_id/eth.total
eth

In [ ]:
eth1 = outcome_yes.groupby(['ethnicity_final'],as_index = False).agg({
    'subject_id': 'nunique'
})
eth1['total'] = 155
eth1['percent'] = eth1.subject_id/eth1.total
eth1

In [ ]:
outcome_no.age_at_diagnosis.mean()

In [ ]:
outcome_yes.age_at_diagnosis.mean()